In [1]:
from datetime import datetime
import datetime as dt
import time
from subgrounds.subgraph import SyntheticField, FieldPath
from subgrounds.subgrounds import Subgrounds
import pandas as pd
import duckdb as db
import seaborn as sns
import matplotlib.pyplot as plt
import IPython as ip
import  os as os

In [2]:
sg = Subgrounds()
apiKey = input('API Key, please')

In [429]:
#for when we save the files
file = input('Selet a folder to save output') ##enter your file path here - the file is in the repo "summary_stats.csv".
file = file+'/data'
if not os.path.exists(file):
    os.makedirs(file)

*******************************************************************
*******************************************************************
LET'S LOOK AT SILO BORROWS FIRST

In [901]:
#SILO subgraph: https://gateway.thegraph.com/api/[api-key]/subgraphs/id/6rcT2GzBXGoi3ZMus92LHajM2b9qWGj7Z3dDhPxoUogB
silo = sg.load_api('https://gateway.thegraph.com/api/'+apiKey+'/subgraphs/id/6rcT2GzBXGoi3ZMus92LHajM2b9qWGj7Z3dDhPxoUogB')

##this captures the list of people who borrowed XAI against gOHM
borrows = silo.Query.borrows(
    orderBy='timestamp',
    orderDirection='desc',
    first=1000,
    #skip = skipValue,
    where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' }

)

borrowsDb = sg.query_df([
    borrows.id
    ,borrows.hash
    ,borrows.logIndex
    ,borrows.blockNumber
    ,borrows.timestamp
    ,borrows.account
    ,borrows.market
    ,borrows.market.rates
    ,borrows.asset
    ,borrows.amount
    ,borrows.amountUSD 
])
borrowsDb.head(100)

#this, in turn, pulls repayment events
repays = silo.Query.repays(
    orderBy='timestamp',
    orderDirection='desc',
    first=1000,
    #skip = skipValue,
    where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' }

)

repaysDb = sg.query_df([
    repays.id
    ,repays.hash
    ,repays.logIndex
    ,repays.blockNumber
    ,repays.timestamp
    ,repays.account
    ,repays.market
    ,repays.market.rates
    ,repays.asset
    ,repays.amount
    ,repays.amountUSD
    ,repays.position
])
repaysDb.head(100)

In [914]:
oldSiloBorrowsDb = pd.DataFrame()
siloFileName = file+'/siloFinanceGohmBorrows.csv'
try:
    oldSiloBorrowsDb = pd.read_csv (siloFileName)
except:
    oldSiloBorrowsDb = pd.DataFrame()
oldSiloBorrowsDb.head(100)

""


In [915]:
maxSiloBorrowTimestamp = 0
try:
    maxSiloBorrowTimestamp = (max(0,oldSiloBorrowsDb['transaction_timestamp'].max()))-5
except:
        maxSiloBorrowTimestamp = 0
maxSiloBorrowTimestamp = int(maxSiloBorrowTimestamp)
maxSiloBorrowTimestamp

0

In [916]:
# 0xc413dd03555f3eb29d834b482d386b2999dc2eb0

In [917]:
##this captures the ENTIRE list of people who borrowed stablecoins against gOHM
borrowTicker = 0
borrowslist = pd.DataFrame()
siloBorrowsDb = pd.DataFrame()
borrowListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (borrowTicker)*(1000)
    borrows = silo.Query.borrows(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0'
                ,'timestamp_gte': maxSiloBorrowTimestamp
                } #gOHM market
    )

    borrowslist = sg.query_df([
        borrows.id
        ,borrows.hash
        ,borrows.blockNumber
        ,borrows.timestamp
        ,borrows.account.id
        ,borrows.market.id
        ,borrows.market.protocol.id
        ,borrows.market.rates.rate
        ,borrows.market.rates.duration
        ,borrows.market.rates.side
        ,borrows.market.rates.type
        ,borrows.market.rates.token.id
        ,borrows.asset.name
        ,borrows.asset.symbol
        ,borrows.asset.decimals
        ,borrows.asset.lastPriceUSD
        ,borrows.amount
        ,borrows.amountUSD
    ])

    siloBorrowsDb=pd.concat([siloBorrowsDb, borrowslist])
    recordTimestamp1 = siloBorrowsDb.iat[borrowTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    siloBorrowsDbLength = len(siloBorrowsDb)
    borrowListLength = len(borrowslist)
    recordID = siloBorrowsDb.iat[borrowTicker,0]
    borrow = siloBorrowsDb.iat[borrowTicker,1]
    print("iterations: ", borrowTicker, "Lines skipped: ",skipValue, "records collected: ", borrowListLength, " - latest recordID: ",borrow, " ", recordID , " - borrows DB length: ", siloBorrowsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", borrowTicker, "records collected: ", borrowListLength, " - userdb length: ", borrowsDbLength)
    borrowTicker = borrowTicker+1

    if daysAgo>daysLimit: exit =True
    if borrowListLength<1000: exit =True

siloBorrowsDb.columns = siloBorrowsDb.columns.str.replace("borrows", "transaction")
siloBorrowsDb['event']='borrow'
siloBorrowsDb.drop_duplicates
print("Done. Total Records collected: ", siloBorrowsDbLength)

iterations:  0 Lines skipped:  0 records collected:  156  - latest recordID:  0x66fe10ed6dcb8b83bdb5b0c46cd321060734b9063bd5ff4b67d962f8a7b60152   0x66fe10ed6dcb8b83bdb5b0c46cd321060734b9063bd5ff4b67d962f8a7b60152-239  - borrows DB length:  156  - latest record from:  2023-02-17 14:39:23 2 days ago
Done. Total Records collected:  156


In [918]:
siloBorrowsDb['transactionReadableAmount'] = siloBorrowsDb['transaction_amount']/(10**(siloBorrowsDb['transaction_asset_decimals']))
siloBorrowsDb['transactionReadableAmount']=siloBorrowsDb['transactionReadableAmount'].astype('float')
siloBorrowsDb['dateTime'] =(pd.to_datetime(siloBorrowsDb['transaction_timestamp'],unit='s'))

siloBorrowEventsDbTimedClean = siloBorrowsDb[['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime']].copy()
siloBorrowEventsDbTimedClean.sort_values(by=['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime'])

siloBorrowEventsDbTimedClean.drop_duplicates(subset=['transaction_hash'],keep='first', inplace=True)
siloBorrowEventsDbTimedClean['calDate'] = siloBorrowEventsDbTimedClean['dateTime'].dt.normalize()
siloBorrowEventsDbTimedClean.head(100)

,event,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_asset_name,transaction_asset_symbol,transactionReadableAmount,transaction_asset_lastPriceUSD,transaction_amountUSD,dateTime,calDate
0,borrow,0x66fe10ed6dcb8b83bdb5b0c46cd321060734b9063bd5...,16651434,1676673563,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,60500.000000,0.992017,59947.550425,2023-02-17 22:39:23,2023-02-17
6,borrow,0xd9329978828295f4f091eaa77a6d8c388c5616bbe92c...,16644592,1676590307,0x96035b45bc0d22f0ed11283a91145fb001c48370,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Wrapped Ether,WETH,0.600000,1674.121561,995.052978,2023-02-16 23:31:47,2023-02-16
12,borrow,0x3b0f26e10c63a71c916e8a68bd1889d9bab38aeb62d7...,16641746,1676556011,0xee95e4cf086fc80df7ae0f39dfc9ea53a3eaadcb,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,100000.000000,0.992017,100012.487283,2023-02-16 14:00:11,2023-02-16
18,borrow,0xfa292276fc71f90e29ca90a7b8a2be9a4094f459fb99...,16634870,1676472911,0x16ff8ad104fa6195260a6d4a3f311c5f120df77d,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,3000.000000,0.992017,3023.153053,2023-02-15 14:55:11,2023-02-15
24,borrow,0xd5dc7147192591c7d19856d12e9278928932a5921653...,16627480,1676383655,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,1000.000000,0.992017,992.079025,2023-02-14 14:07:35,2023-02-14
30,borrow,0x39c11cb01e34d9f8ca55a8820482eaf8a217033cdb6f...,16626801,1676375459,0x16ff8ad104fa6195260a6d4a3f311c5f120df77d,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Wrapped Ether,WETH,1.920000,1674.121561,2885.217480,2023-02-14 11:50:59,2023-02-14
36,borrow,0xf1d5b9c7f23bd9935061e4d08734cbfba9b9a53bacd5...,16626396,1676370563,0x339fd7c17ce670e7821f2f2ee94ddd49771483da,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,4000.000000,0.992017,3963.627622,2023-02-14 10:29:23,2023-02-14
42,borrow,0x97811f7e7b7d202797093d44047ed986054be3dd6120...,16623063,1676330255,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,2457.516074,0.992017,2446.717700,2023-02-13 23:17:35,2023-02-13
48,borrow,0x3dba2dd3c913b399164efd71db73121c9c94c0096139...,16622907,1676328371,0x46535cbc3fa574c32ea392d4f89c518290597072,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,26.400000,2809.191078,71910.521424,2023-02-13 22:46:11,2023-02-13
54,borrow,0xbcfc05398a874e98a80e823ffb016c119b06c7b07a7c...,16619977,1676292983,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,XAI Stablecoin,XAI,3090.000000,0.992017,3085.406589,2023-02-13 12:56:23,2023-02-13


In [921]:
path =file+'/siloFinanceGohmBorrows.csv'
siloBorrowEventsDbTimedClean.to_csv(path, index = False)

path =file+'/siloFinanceGohmBorrows_piped.csv'
siloBorrowEventsDbTimedClean.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(siloBorrowEventsDbTimedClean),' records')

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\gohmAsStablecoinCollateral/data/siloFinanceGohmBorrows_piped.csv file saved, 26  records


In [1068]:
totalBorrowedUSDSilo = sum(siloBorrowEventsDbTimedClean['transaction_amountUSD'])
print('A total of $',round(totalBorrowedUSDSilo,2),  'has been borrowed in different stablecoins over time over at Silo')

A total of $ 564579.7 has been borrowed in different stablecoins over time over at Silo


In [928]:
gOHMBorrowedSilo = db.query("Select distinct "
                           "transaction_hash "
                           ",transaction_timestamp "
                           ", dateTime "
                           ",transaction_account_id "
                           ", transaction_asset_symbol "
                           ", transactionReadableAmount "
                           ", transaction_amountUSD "
                           "from siloBorrowsDb where lower(transaction_asset_symbol) = 'gohm'").df()

gOHMBorrowedSilo.head(10)

,transaction_hash,transaction_timestamp,dateTime,transaction_account_id,transaction_asset_symbol,transactionReadableAmount,transaction_amountUSD
0,0x3dba2dd3c913b399164efd71db73121c9c94c0096139...,1676328371,2023-02-13 22:46:11,0x46535cbc3fa574c32ea392d4f89c518290597072,gOHM,26.400,71910.521424
1,0x83addaa30259315b10869fa8653a468eb1cc82a43f57...,1669229867,2022-11-23 18:57:47,0x7776421dd146dfd6ea947cbf29b69c519807d259,gOHM,0.182,399.040935


In [1067]:
totalSiloBorrowedgOHM = sum(ohmBorrowedSilo['transactionReadableAmount'])
totalSiloBorrowedgOHMinUSD = sum(ohmBorrowedSilo['transaction_amountUSD'])
print('A total of ',round(totalSiloBorrowedgOHM,2),  'OHM has been borrowed in different stablecoins over time in Silo, roughly USD$',totalSiloBorrowedgOHMinUSD)

A total of  26.58 OHM has been borrowed in different stablecoins over time in Silo, roughly USD$ 72309.56235872359


In [930]:
path =file+'/ohmBorrowedSilo.csv'
ohmBorrowedSilo.to_csv(path, index = False)

path =file+'/ohmBorrowedSilo_piped.csv'
ohmBorrowedSilo.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(ohmBorrowedSilo),' records')

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\gohmAsStablecoinCollateral/data/ohmBorrowedSilo_piped.csv file saved, 2  records


********************************************************************
********************************************************************
SILO COLLATERAL DEPOSITS

In [931]:
oldSiloDepositsDb = pd.DataFrame()
siloFileName = file+'/siloFinanceGohmDeposits.csv'
try:
    oldSiloDepositsDb = pd.read_csv (siloFileName)
except:
    oldSiloDepositsDb = pd.DataFrame()
oldSiloDepositsDb.head(100)

""


In [932]:
oldSiloDepositsDb['transaction_timestamp'].max()

KeyError: 'transaction_timestamp'

In [934]:
maxSiloDepositsTimestamp = 0
try:
    maxSiloDepositsTimestamp = (max(0,oldSiloDepositsDb['transaction_timestamp'].max()))-1
except:
        maxSiloDepositsTimestamp = 0
maxSiloDepositsTimestamp = int(maxSiloDepositsTimestamp)
maxSiloDepositsTimestamp

0

In [936]:
##this captures the ENTIRE list of people who deposited OHM
depositTicker = 0
depositslist = pd.DataFrame()
siloDepositsDb = pd.DataFrame()
depositListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (depositTicker)*(1000)
    deposits = silo.Query.deposits(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0'
                ,'timestamp_gte': maxSiloDepositsTimestamp
                } #gOHM market
    )

    depositslist = sg.query_df([
        deposits.id
        ,deposits.hash
        ,deposits.blockNumber
        ,deposits.timestamp
        ,deposits.account.id
        ,deposits.market.id
        ,deposits.market.protocol.id
        ,deposits.market.rates.rate
        ,deposits.market.rates.duration
        ,deposits.market.rates.side
        ,deposits.market.rates.type
        ,deposits.market.rates.token.id
        ,deposits.asset.name
        ,deposits.asset.symbol
        ,deposits.asset.decimals
        ,deposits.asset.lastPriceUSD
        ,deposits.amount
        ,deposits.amountUSD
    ])

    siloDepositsDb=pd.concat([siloDepositsDb, depositslist])
    recordTimestamp1 = siloDepositsDb.iat[depositTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    siloDepositsDbLength = len(siloDepositsDb)
    siloDepositListLength = len(depositslist)
    recordID = siloDepositsDb.iat[depositTicker,0]
    deposit = siloDepositsDb.iat[depositTicker,1]
    print("iterations: ", depositTicker, "Lines skipped: ",skipValue, "records collected: ", siloDepositListLength, " - latest recordID: ",deposit, " ", recordID , " - deposits DB length: ", siloDepositsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", depositTicker, "records collected: ", depositListLength, " - userdb length: ", depositsDbLength)
    depositTicker = depositTicker+1

    if daysAgo>daysLimit: exit =True
    if depositListLength<1000: exit =True

siloDepositsDb.columns = siloDepositsDb.columns.str.replace("deposits", "transaction")
siloDepositsDb['event']='deposit'
siloDepositsDb.drop_duplicates
print("Done. Total Records collected: ", siloDepositsDbLength)

iterations:  0 Lines skipped:  0 records collected:  294  - latest recordID:  0xecd6140bbb896413a25d34ca87f5503b56c444f1e582ce06f89b5eb395fa6a39   0xecd6140bbb896413a25d34ca87f5503b56c444f1e582ce06f89b5eb395fa6a39-144  - deposits DB length:  294  - latest record from:  2023-02-17 14:34:23 2 days ago
iterations:  1 Lines skipped:  1000 records collected:  0  - latest recordID:  0xecd6140bbb896413a25d34ca87f5503b56c444f1e582ce06f89b5eb395fa6a39   0xecd6140bbb896413a25d34ca87f5503b56c444f1e582ce06f89b5eb395fa6a39-144  - deposits DB length:  294  - latest record from:  2023-02-17 14:34:23 2 days ago
iterations:  2 Lines skipped:  2000 records collected:  0  - latest recordID:  0xecd6140bbb896413a25d34ca87f5503b56c444f1e582ce06f89b5eb395fa6a39   0xecd6140bbb896413a25d34ca87f5503b56c444f1e582ce06f89b5eb395fa6a39-144  - deposits DB length:  294  - latest record from:  2023-02-17 14:34:23 2 days ago
iterations:  3 Lines skipped:  3000 records collected:  0  - latest recordID:  0xecd6140bbb8964

In [937]:
siloDepositsDb.head(10)

,transaction_id,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_market_protocol_id,transaction_market_rates_rate,transaction_market_rates_duration,transaction_market_rates_side,transaction_market_rates_type,transaction_market_rates_token_id,transaction_asset_name,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_amount,transaction_amountUSD,event
0,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,16651409,1676673263,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,VARIABLE,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,gOHM,18,2838.193286,48922199130466954750,138354.126796,deposit
1,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,16651409,1676673263,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.983536,None,BORROWER,VARIABLE,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Governance OHM,gOHM,18,2838.193286,48922199130466954750,138354.126796,deposit
2,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,16651409,1676673263,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,2.840751,None,BORROWER,VARIABLE,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,Governance OHM,gOHM,18,2838.193286,48922199130466954750,138354.126796,deposit
3,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,16651409,1676673263,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,LENDER,VARIABLE,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,gOHM,18,2838.193286,48922199130466954750,138354.126796,deposit
4,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,16651409,1676673263,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.117764,None,LENDER,VARIABLE,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Governance OHM,gOHM,18,2838.193286,48922199130466954750,138354.126796,deposit
5,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,16651409,1676673263,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,1.614002,None,LENDER,VARIABLE,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,Governance OHM,gOHM,18,2838.193286,48922199130466954750,138354.126796,deposit
6,0x54cd3f5374780bceaffae0661e2688eeca47df22b8aa...,0x54cd3f5374780bceaffae0661e2688eeca47df22b8aa...,16641740,1676555939,0xee95e4cf086fc80df7ae0f39dfc9ea53a3eaadcb,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,VARIABLE,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,gOHM,18,2838.193286,50749991834761030902,145833.530065,deposit
7,0x54cd3f5374780bceaffae0661e2688eeca47df22b8aa...,0x54cd3f5374780bceaffae0661e2688eeca47df22b8aa...,16641740,1676555939,0xee95e4cf086fc80df7ae0f39dfc9ea53a3eaadcb,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.983536,None,BORROWER,VARIABLE,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Governance OHM,gOHM,18,2838.193286,50749991834761030902,145833.530065,deposit
8,0x54cd3f5374780bceaffae0661e2688eeca47df22b8aa...,0x54cd3f5374780bceaffae0661e2688eeca47df22b8aa...,16641740,1676555939,0xee95e4cf086fc80df7ae0f39dfc9ea53a3eaadcb,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,2.840751,None,BORROWER,VARIABLE,0xd7c9f0e536dc865ae858b0c

In [940]:
siloDepositsDb['event'] ='deposit'
siloDepositsDb['transactionReadableAmount'] = siloDepositsDb['transaction_amount']/(10**(siloDepositsDb['transaction_asset_decimals']))
siloDepositsDb['transactionReadableAmount']=siloDepositsDb['transactionReadableAmount'].astype('float')
siloDepositsDb['dateTime'] =(pd.to_datetime(siloDepositsDb['transaction_timestamp'],unit='s'))


#we only want deposits in GOHM from those that borrowed, so drop anyone that is not on that side
siloDepositsDb.drop(siloDepositsDb.loc[siloDepositsDb['transaction_market_rates_side']!='BORROWER'].index, inplace=True)
siloDepositsDb.drop(siloDepositsDb.loc[siloDepositsDb['transaction_asset_symbol']!='gOHM'].index, inplace=True)


siloDepositEventsDbTimedClean = siloDepositsDb[['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime']].copy()
siloDepositEventsDbTimedClean.sort_values(by=['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime'])
siloDepositEventsDbTimedClean.drop_duplicates(subset=['transaction_hash'],keep='first', inplace=True)
siloDepositEventsDbTimedClean['calDate'] = siloDepositEventsDbTimedClean['dateTime'].dt.normalize()
siloDepositEventsDbTimedClean.head(1000)

,event,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_asset_name,transaction_asset_symbol,transactionReadableAmount,transaction_asset_lastPriceUSD,transaction_amountUSD,dateTime,calDate
0,deposit,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,16651409,1676673263,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,48.922199,2838.193286,138354.126796,2023-02-17 22:34:23,2023-02-17
6,deposit,0x54cd3f5374780bceaffae0661e2688eeca47df22b8aa...,16641740,1676555939,0xee95e4cf086fc80df7ae0f39dfc9ea53a3eaadcb,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,50.749992,2838.193286,145833.530065,2023-02-16 13:58:59,2023-02-16
30,deposit,0xe5163797f0922e4bded89126a213296a8c3d270a39e2...,16627499,1676383883,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,0.180346,2838.193286,490.375269,2023-02-14 14:11:23,2023-02-14
36,deposit,0xcc5e09e1e5c403b6925fa25f0410388de72ef1b45f30...,16627469,1676383523,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,0.663108,2838.193286,1803.040025,2023-02-14 14:05:23,2023-02-14
42,deposit,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,16623070,1676330339,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,0.892669,2838.193286,2431.527276,2023-02-13 23:18:59,2023-02-13
48,deposit,0x3069fa3e9834fd75b4eaae5d1ef1a7910b112ed68e5f...,16623049,1676330087,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,1.087799,2838.193286,2963.036715,2023-02-13 23:14:47,2023-02-13
60,deposit,0x17d879ec97730987652b5bcb00aaf6115b7dd232aaef...,16619969,1676292887,0xc0b1f200aa9546ce074397bed7438bb2ec033877,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,1.812154,2838.193286,4846.273689,2023-02-13 12:54:47,2023-02-13
66,deposit,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,16601490,1676069891,0xa7daeb7fc210f934508c8c7ce833a2cdd6630fb3,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,0.137814,2838.193286,363.447391,2023-02-10 22:58:11,2023-02-10
78,deposit,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,16590634,1675938899,0x51bc3dd10a9ee4946f961038cf5d50057e49eb77,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,9.990000,2838.193286,26839.810526,2023-02-09 10:34:59,2023-02-09
84,deposit,0x204aee0831212791916d6ed3a2c24b5ac2a474743713...,16589368,1675923587,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,119.000000,2838.193286,315110.337935,2023-02-09 06:19:47,2023-02-09


In [941]:
path =file+'/siloFinanceGohmDeposits.csv'
siloDepositEventsDbTimedClean.to_csv(path, index = False)

path =file+'/siloFinanceGohmDeposits_piped.csv'
siloDepositEventsDbTimedClean.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(siloDepositEventsDbTimedClean),' records')

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\gohmAsStablecoinCollateral/data/siloFinanceGohmDeposits_piped.csv file saved, 33  records


In [943]:
totalDeposited = sum(siloDepositEventsDbTimedClean['transactionReadableAmount'])
totalUSDDeposited = sum(siloDepositEventsDbTimedClean['transaction_amountUSD'])
print('There has been a total of',round(totalDeposited,2),  'gOHM deposited cumulatively as collateral  to date, about USD$',totalUSDDeposited)

There has been a total of 364.48 gOHM deposited cumulatively as collateral  to date, about USD$ 976669.3975413579


********************************************************************
********************************************************************
Now we need withdrawals to counter the deposits and see current state

In [946]:
oldSiloWithdrawsDb = pd.DataFrame()
siloFileName = file+'/siloFinanceGohmWithdraws.csv'
try:
    oldSiloWithdrawsDb = pd.read_csv (siloFileName)
except:
    oldSiloWithdrawsDb = pd.DataFrame()
oldSiloWithdrawsDb.head(100)

""


In [947]:
maxSiloWithdrawsTimestamp = 0
try:
    maxSiloWithdrawsTimestamp = (max(0,oldSiloWithdrawsDb['transaction_timestamp'].max()))-1
except:
        maxSiloWithdrawsTimestamp = 0
maxSiloWithdrawsTimestamp = int(maxSiloWithdrawsTimestamp)
maxSiloWithdrawsTimestamp

0

In [948]:
##this captures the ENTIRE list of people who withdrawed OHM
withdrawTicker = 0
withdrawslist = pd.DataFrame()
siloWithdrawsDb = pd.DataFrame()
withdrawListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (withdrawTicker)*(1000)
    withdraws = silo.Query.withdraws(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0'
                ,'timestamp_gte': maxSiloWithdrawsTimestamp
                } #gOHM market
    )

    withdrawslist = sg.query_df([
        withdraws.id
        ,withdraws.hash
        ,withdraws.blockNumber
        ,withdraws.timestamp
        ,withdraws.account.id
        ,withdraws.market.id
        ,withdraws.market.protocol.id
        ,withdraws.market.rates.rate
        ,withdraws.market.rates.duration
        ,withdraws.market.rates.side
        ,withdraws.market.rates.type
        ,withdraws.market.rates.token.id
        ,withdraws.asset.name
        ,withdraws.asset.symbol
        ,withdraws.asset.decimals
        ,withdraws.asset.lastPriceUSD
        ,withdraws.amount
        ,withdraws.amountUSD
    ])

    siloWithdrawsDb=pd.concat([siloWithdrawsDb, withdrawslist])
    recordTimestamp1 = siloWithdrawsDb.iat[withdrawTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    silowithdrawsDbLength = len(siloWithdrawsDb)
    silowithdrawListLength = len(withdrawslist)
    recordID = siloWithdrawsDb.iat[withdrawTicker,0]
    withdraw = siloWithdrawsDb.iat[withdrawTicker,1]
    print("iterations: ", withdrawTicker, "Lines skipped: ",skipValue, "records collected: ", silowithdrawListLength, " - latest recordID: ",withdraw, " ", recordID , " - withdraws DB length: ", silowithdrawsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", withdrawTicker, "records collected: ", withdrawListLength, " - userdb length: ", withdrawsDbLength)
    withdrawTicker = withdrawTicker+1

    if daysAgo>daysLimit: exit =True
    if silowithdrawListLength<1000: exit =True

siloWithdrawsDb.columns = siloWithdrawsDb.columns.str.replace("withdraws", "transaction")
siloWithdrawsDb['event']='withdraw'
siloWithdrawsDb.drop_duplicates
print("Done. Total Records collected: ", silowithdrawsDbLength)

iterations:  0 Lines skipped:  0 records collected:  144  - latest recordID:  0x6874260982e867567e9455254ce574aba7f95db4d6f82bc2485bd1b40a71a83e   0x6874260982e867567e9455254ce574aba7f95db4d6f82bc2485bd1b40a71a83e-347  - withdraws DB length:  144  - latest record from:  2023-02-15 14:58:59 4 days ago
Done. Total Records collected:  144


In [949]:
siloWithdrawEventsDbTimedClean = pd.DataFrame()
siloWithdrawsDb['transactionReadableAmount'] = siloWithdrawsDb['transaction_amount']/(10**(siloWithdrawsDb['transaction_asset_decimals']))
siloWithdrawsDb['transactionReadableAmount']=siloWithdrawsDb['transactionReadableAmount'].astype('float')
siloWithdrawsDb['dateTime'] =(pd.to_datetime(siloWithdrawsDb['transaction_timestamp'],unit='s'))

#we only want withdrawals in GOHM from those that borrowed, so drop anyone that is not on that side
siloWithdrawsDb.drop(siloWithdrawsDb.loc[siloWithdrawsDb['transaction_market_rates_side']!='BORROWER'].index, inplace=True)
siloWithdrawsDb.drop(siloWithdrawsDb.loc[siloWithdrawsDb['transaction_asset_symbol']!='gOHM'].index, inplace=True)

siloWithdrawEventsDbTimedClean = siloWithdrawsDb[['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime']].copy()
siloWithdrawEventsDbTimedClean.sort_values(by=['event','transaction_hash','transaction_blockNumber','transaction_timestamp','transaction_account_id','transaction_market_id','transaction_asset_name','transaction_asset_symbol', 'transactionReadableAmount','transaction_asset_lastPriceUSD','transaction_amountUSD','dateTime'])
siloWithdrawEventsDbTimedClean.drop_duplicates(subset=['transaction_hash'],keep='first', inplace=True)
siloWithdrawEventsDbTimedClean['calDate'] = siloWithdrawEventsDbTimedClean['dateTime'].dt.normalize()
siloWithdrawsDb.head(100)

,transaction_id,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_market_protocol_id,transaction_market_rates_rate,transaction_market_rates_duration,transaction_market_rates_side,...,transaction_market_rates_token_id,transaction_asset_name,transaction_asset_symbol,transaction_asset_decimals,transaction_asset_lastPriceUSD,transaction_amount,transaction_amountUSD,event,transactionReadableAmount,dateTime
6,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,16635032,1676474867,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,gOHM,18,2838.193286,25000038820033231117,71148.084004,withdraw,25.000039,2023-02-15 15:27:47
7,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,16635032,1676474867,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.983536,None,BORROWER,...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Governance OHM,gOHM,18,2838.193286,25000038820033231117,71148.084004,withdraw,25.000039,2023-02-15 15:27:47
8,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,16635032,1676474867,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,2.840751,None,BORROWER,...,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,Governance OHM,gOHM,18,2838.193286,25000038820033231117,71148.084004,withdraw,25.000039,2023-02-15 15:27:47
12,0x85b689883076aa92af0a2356f8e4084610578bb48590...,0x85b689883076aa92af0a2356f8e4084610578bb48590...,16634844,1676472599,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,gOHM,18,2838.193286,64393035133917233507,183257.358356,withdraw,64.393035,2023-02-15 14:49:59
13,0x85b689883076aa92af0a2356f8e4084610578bb48590...,0x85b689883076aa92af0a2356f8e4084610578bb48590...,16634844,1676472599,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.983536,None,BORROWER,...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Governance OHM,gOHM,18,2838.193286,64393035133917233507,183257.358356,withdraw,64.393035,2023-02-15 14:49:59
14,0x85b689883076aa92af0a2356f8e4084610578bb48590...,0x85b689883076aa92af0a2356f8e4084610578bb48590...,16634844,1676472599,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,2.840751,None,BORROWER,...,0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac,Governance OHM,gOHM,18,2838.193286,64393035133917233507,183257.358356,withdraw,64.393035,2023-02-15 14:49:59
18,0xd931b79cfc34df088f325e8adde9122af7ad64a0ba30...,0xd931b79cfc34df088f325e8adde9122af7ad64a0ba30...,16628161,1676391839,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.000000,None,BORROWER,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Governance OHM,gOHM,18,2838.193286,36000000000000000000,100991.276713,withdraw,36.000000,2023-02-14 16:23:59
19,0xd931b79cfc34df088f325e8adde9122af7ad64a0ba30...,0xd931b79cfc34df088f325e8adde9122af7ad64a0ba30...,16628161,1676391839,0xb0399dac8f4d4100b49b2a7b3873481114229d18,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,0.983536,None,BORROWER,...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Governance OHM,gOHM,18,2838.193286,36000000000000000000,100991.276713,withdraw,36.000000,2023-02-14 16:23:59
20,0xd931b79cfc34df088f325e8adde9122af7ad64a0ba30...,0xd931b79cfc34df088f325e8adde9

In [950]:
path =file+'/siloFinanceGohmWithdraws.csv'
siloWithdrawEventsDbTimedClean.to_csv(path, index = False)

path =file+'/siloFinanceGohmWithdraws_piped.csv'
siloWithdrawEventsDbTimedClean.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(siloWithdrawEventsDbTimedClean),' records')

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\gohmAsStablecoinCollateral/data/siloFinanceGohmWithdraws_piped.csv file saved, 12  records


In [951]:
totalWithdrawed = sum(siloWithdrawEventsDbTimedClean['transactionReadableAmount'])
print('There has been a total of',round(totalWithdrawed,2),  'OHM withdrawn cumulatively as collateral  to date')
totalWithdrawedUSD =sum(siloWithdrawEventsDbTimedClean['transaction_amountUSD'])
print('That is a value of USD$',round(totalWithdrawedUSD,2))

There has been a total of 182.18 OHM withdrawn cumulatively as collateral  to date
That is a value of USD$ 502805.15


In [952]:
siloCollateralBalance = totalDeposited -(totalWithdrawed)
print(totalDeposited, ' - ',totalWithdrawed,'=', siloCollateralBalance, 'currently deposited as collateral in Silo' )

364.4811867772614  -  182.17769420239753 = 182.3034925748639 currently deposited as collateral in Silo


********************************************************************
********************************************************************
VESTA BORROWS - Their API is poo RN

In [ ]:
#https://api.thegraph.com/subgraphs/name/corerouter/vesta-finance
vesta = sg.load_api('https://api.thegraph.com/subgraphs/name/shinitakunai/vestafinance-v1_2')

In [ ]:
oldVestaBorrowsDb = pd.DataFrame()
siloFileName = file+'/vesta_borrow_events.csv'
try:
    oldVestaBorrowsDb = pd.read_csv (siloFileName)
except:
    oldVestaBorrowsDb = pd.DataFrame()
oldVestaBorrowsDb.head(100)

In [ ]:
maxVestaBorrowTimestamp = 0
try:
    maxVestaTroveChangeId = (max(0,oldVestaBorrowsDb['troveChanges_id'].max()))
    maxVestaTroveChangeId = int(maxVestaTroveChangeId)
    vestaBorrows = vesta.Query.troveChanges(
        orderBy='id',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where= { 'asset': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1'  #gOHM market
                ,'debtChange_lt':0
                ,'id_gte': maxVestaTroveChangeId
         }
        )
    vestaBorrowslist = sg.query_df([
            vestaBorrows.id
            ,vestaBorrows.transaction.id
            ,vestaBorrows.transaction.blockNumber
            ,vestaBorrows.debtChange
            ,vestaBorrows.transaction.timestamp
            ,vestaBorrows.trove.owner.id
            ,vestaBorrows.trove.asset.id
            ,vestaBorrows.trove.asset.name
        ])
except:
    maxVestaTroveChangeId = 0
    vestaBorrows = vesta.Query.troveChanges(
        orderBy='id',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where= { 'asset': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1'  #gOHM market
                ,'debtChange_lt':0
               #,'id_gte': maxVestaTroveChangeId
         }
    )
    vestaBorrowslist = sg.query_df([
            vestaBorrows.id
            ,vestaBorrows.transaction.id
            ,vestaBorrows.transaction.blockNumber
            ,vestaBorrows.debtChange
            ,vestaBorrows.transaction.timestamp
            ,vestaBorrows.trove.owner.id
            ,vestaBorrows.trove.asset.id
            ,vestaBorrows.trove.asset.name
        ])

vestaBorrowslist.head(10)

vestaBorrows = vesta.Query.troveChanges(
        orderBy='id',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where= { 'asset': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1'  #gOHM market
                ,'debtChange_lt':0
                ,'id_gte': maxVestaTroveChangeId
         }
    )
vestaBorrowslist = sg.query_df([
        vestaBorrows.id
        ,vestaBorrows.transaction.id
        ,vestaBorrows.transaction.blockNumber
        ,vestaBorrows.debtChange
        ,vestaBorrows.transaction.timestamp
        ,vestaBorrows.trove.owner.id
        ,vestaBorrows.trove.asset.id
        ,vestaBorrows.trove.asset.name
    ])

vestaBorrowslist.head(10)

In [ ]:
##this captures the list of people who vestaBorrowed XAI against gOHM
vestaBorrowTicker = 0
vestaBorrowslist = pd.DataFrame()
vestaBorrowsDb = pd.DataFrame()
vestaBorrowListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (vestaBorrowTicker)*(1000)
    try:
        maxVestaTroveChangeId = (max(0,oldVestaBorrowsDb['troveChanges_id'].max()))
        maxVestaTroveChangeId = int(maxVestaTroveChangeId)
        vestaBorrows = vesta.Query.troveChanges(
            orderBy='id',
            orderDirection='desc',
            first=1000,
            skip = skipValue,
            where= { 'asset': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1'  #gOHM market
                    ,'debtChange_gt':0
                    ,'id_gte': maxVestaTroveChangeId
             }
            )
        vestaBorrowslist = sg.query_df([
                vestaBorrows.id
                ,vestaBorrows.transaction.id
                ,vestaBorrows.transaction.blockNumber
                ,vestaBorrows.debtChange
                ,vestaBorrows.transaction.timestamp
                ,vestaBorrows.trove.owner.id
                ,vestaBorrows.trove.asset.id
                ,vestaBorrows.trove.asset.name
            ])
    except:
        maxVestaTroveChangeId = 0
        vestaBorrows = vesta.Query.troveChanges(
            orderBy='id',
            orderDirection='desc',
            first=1000,
            skip = skipValue,
            where= { 'asset': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1'  #gOHM market
                    ,'debtChange_gt':0
                   #,'id_gte': maxVestaTroveChangeId
             }
        )
        vestaBorrowslist = sg.query_df([
                vestaBorrows.id
                ,vestaBorrows.transaction.id
                ,vestaBorrows.transaction.blockNumber
                ,vestaBorrows.debtChange
                ,vestaBorrows.transaction.timestamp
                ,vestaBorrows.trove.owner.id
                ,vestaBorrows.trove.asset.id
                ,vestaBorrows.trove.asset.name
            ])

    #vestaBorrowslist.sort_values(by=['vestaBorrows_timestamp'])

    vestaBorrowsDb=pd.concat([vestaBorrowsDb, vestaBorrowslist])
    recordTimestamp1 = vestaBorrowsDb.iat[vestaBorrowTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    vestaBorrowsDbLength = len(vestaBorrowsDb)
    vestaBorrowListLength = len(vestaBorrowslist)
    recordID = vestaBorrowsDb.iat[vestaBorrowTicker,0]
    vestaBorrow = vestaBorrowsDb.iat[vestaBorrowTicker,1]
    print("iterations: ", vestaBorrowTicker, "Lines skipped: ",skipValue, "records collected: ", vestaBorrowListLength, " - latest recordID: ",vestaBorrow, " ", recordID , " - vestaBorrows DB length: ", vestaBorrowsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", vestaBorrowTicker, "records collected: ", vestaBorrowListLength, " - userdb length: ", vestaBorrowsDbLength)
    vestaBorrowTicker = vestaBorrowTicker+1

    if daysAgo>daysLimit: exit =True
    if vestaBorrowListLength<1000: exit =True

#vestaBorrowsDbTimed.columns = vestaBorrowsDbTimed.columns.str.replace("borrows", "transaction")
vestaBorrowsDb['event']='borrow'
print("Done. Total Records collected: ", vestaBorrowsDbLength)

In [ ]:
vestaBorrowsDbTimed = db.query("select "
               "    * "
                    " ,to_timestamp(troveChanges_transaction_timestamp::int) dateTime "
               "From vestaBorrowsDb "
                               "order by troveChanges_transaction_timestamp desc"
               ).df()

vestaBorrowsDbTimed.head(1000)

In [ ]:
vestaBorrowsDbTimed = pd.concat([oldVestaBorrowsDb,vestaBorrowsDbTimed]).drop_duplicates().reset_index(drop=True)
vestaBorrowsDbTimed.drop_duplicates()
vestaBorrowsDbTimed = db.query("select "
               "    * "
               "From vestaBorrowsDbTimed "
                         "group by 1,2,3,4,5,6,7,8,9,10 "
                     # "order by borrows_amountUSD desc "
               #"where lower(liquidityPools_name) like lower('%olympus%') "
                #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
               #" limit 100 "
               ).df()

vestaBorrowsDbTimed.head(100)

In [ ]:
path =file+'/vesta_borrow_events.csv'
vestaBorrowsDbTimed.to_csv(path, index = False)

path =file+'/vesta_borrow_events_piped.csv'
vestaBorrowsDbTimed.to_csv(path, index = False, sep='|')
print(path, 'file saved,', len(vestaBorrowsDbTimed),' records')

************************************************************
FINE.... FRAX TOO

This subgraph is completely different, so we're going to  have to crunch in order to merge all these tables into coherent reports

In [953]:
frax = sg.load_api('https://gateway.thegraph.com/api/'+apiKey+'/subgraphs/id/GELTrjPJYEzxyp6Y2CtEaYpGHcJNrJA6i5Ci4KfJSEsf')
#frax = sg.load_api('https://api.thegraph.com/subgraphs/name/frax-finance-data/fraxlend-subgraph---mainnet')

In [978]:
oldFraxActions = pd.DataFrame()
fraxFileName = file+'/frax_borrows.csv'
try:
    oldFraxActions = pd.read_csv (fraxFileName)
except:
    oldFraxActions = pd.DataFrame()
oldFraxActions.head(1000)

""


In [979]:
maxBorrowFraxTimestamp = 0
try:
    maxBorrowFraxTimestamp = (max(1,oldFraxActions['actions_timestamp'].max()))-1
except:
        maxBorrowFraxTimestamp = 0
maxBorrowFraxTimestamp = int(maxBorrowFraxTimestamp)
maxBorrowFraxTimestamp

0

In [980]:
actionsQuery = frax.Query.actions(
     orderBy='timestamp'
    ,orderDirection='desc'
    ,first=1000
    #,where = [frax.Query.actions.token.id == '0x66bf36dba79d4606039f04b32946a260bcd3ff52']
    ,where={ 'pair': '0x66bf36dba79d4606039f04b32946a260bcd3ff52'
             ,'timestamp_gte': maxBorrowFraxTimestamp
             }
    #,where = {'positions.pairid':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}
#{'users_positions_pair_id':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}

)
actions= sg.query_df([
    actionsQuery.id
    ,actionsQuery.type
    ,actionsQuery.amount
    ,actionsQuery.timestamp
    ,actionsQuery.block
    ,actionsQuery.token.name
    ,actionsQuery.root.address
    ,actionsQuery.pair.asset.symbol
    ,actionsQuery.pair.asset.decimals
    ,actionsQuery.pair.collateral.symbol
    ,actionsQuery.pair.collateral.decimals
    ]
)
actions['dateTime'] =(pd.to_datetime(actions['actions_timestamp'],unit='s'))
actions.head(100)

,actions_id,actions_type,actions_amount,actions_timestamp,actions_block,actions_token_name,actions_root_address,actions_pair_asset_symbol,actions_pair_asset_decimals,actions_pair_collateral_symbol,actions_pair_collateral_decimals,dateTime
0,0xa36fce829a2866df6030c9e52d02c538b7f1ca3ba28a...,borrowAsset,0,1676866727,16667337,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-20 04:18:47
1,0xa36fce829a2866df6030c9e52d02c538b7f1ca3ba28a...,addCollateral,7006979439076347667,1676866727,16667337,Governance OHM,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-20 04:18:47
2,0x374e4347a5abad8ebf70cbb9fde86005a3bfbeaeee44...,borrowAsset,10000000000000000000000,1676865431,16667229,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-20 03:57:11
3,0x4c7cd0a0fe188a8049ac8c28269d52106804986dfb99...,borrowAsset,0,1676856395,16666484,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-20 01:26:35
4,0x4c7cd0a0fe188a8049ac8c28269d52106804986dfb99...,addCollateral,6480000000000000000,1676856395,16666484,Governance OHM,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-20 01:26:35
...,...,...,...,...,...,...,...,...,...,...,...,...
95,0x2dc9bc4f7c7ac1708768d1f6f6e6a41b8fb4964fd894...,borrowAsset,0,1675973579,16593503,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-09 20:12:59
96,0x2dc9bc4f7c7ac1708768d1f6f6e6a41b8fb4964fd894...,addCollateral,1849694947953999686,1675973579,16593503,Governance OHM,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-09 20:12:59
97,0x5dea5a1baef298676a0c47666ca76db0d00637a96750...,borrowAsset,5000000000000000000000,1675972547,16593417,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-09 19:55:47
98,0x2c4738e7f57d0ef25cef1541393689e07f7f01b8774a...,borrowAsset,0,1675971107,16593299,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-09 19:31:47


In [981]:
actionListLength=0
actionTicker = 0
fraxUsersDb = pd.DataFrame()
actionsDb = pd.DataFrame()
actionListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    #listLength=0
    skipValue = (actionTicker)*(1000)
    actionsQuery = frax.Query.actions(
         orderBy='timestamp'
        ,orderDirection='desc'
        ,first=1000
        ,skipValue = skipValue
        #,where = [frax.Query.actions.token.id == '0x66bf36dba79d4606039f04b32946a260bcd3ff52']
        ,where={ 'pair': '0x66bf36dba79d4606039f04b32946a260bcd3ff52'
                 ,'timestamp_gte': maxBorrowFraxTimestamp
                 }
        #,where = {'positions.pairid':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}
    #{'users_positions_pair_id':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}

    )
    actions= sg.query_df([
        actionsQuery.id
        ,actionsQuery.type
        ,actionsQuery.amount
        ,actionsQuery.timestamp
        ,actionsQuery.block
        ,actionsQuery.token.name
        ,actionsQuery.root.address
        ,actionsQuery.pair.asset.symbol
        ,actionsQuery.pair.asset.decimals
        ,actionsQuery.pair.collateral.symbol
        ,actionsQuery.pair.collateral.decimals
        ]
    )

    actions.sort_values(by=['actions_timestamp'])

    actionsDb=pd.concat([actionsDb, actions])
    recordTimestamp1 = actionsDb.iat[actionTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    actionsDbLength = len(actionsDb)
    actionListLength = len(actions)
    recordID = actionsDb.iat[actionTicker,0]
    action = actionsDb.iat[actionTicker,1]
    print("iterations: ", actionTicker, "Lines skipped: ",skipValue, "records collected: ", actionListLength, " - latest recordID: ",action, " ", recordID , " - actions DB length: ", actionsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", actionTicker, "records collected: ", actionListLength, " - userdb length: ", actionsDbLength)
    if daysAgo>daysLimit: exit =True
    if actionListLength<1000: exit =True
    actionTicker = actionTicker+1
actionsDb['DateTime']=(pd.to_datetime(actionsDb['actions_timestamp'],unit='s'))
print("Done. Total Records collected: ", actionsDbLength)

iterations:  0 Lines skipped:  0 records collected:  1000  - latest recordID:  borrowAsset   0xa36fce829a2866df6030c9e52d02c538b7f1ca3ba28a39371eadbd1a7d19c5ec+1  - actions DB length:  1000  - latest record from:  2023-02-19 20:18:47 0 days ago
iterations:  1 Lines skipped:  1000 records collected:  1000  - latest recordID:  addCollateral   0xa36fce829a2866df6030c9e52d02c538b7f1ca3ba28a39371eadbd1a7d19c5ec  - actions DB length:  2000  - latest record from:  2023-02-19 20:18:47 0 days ago
iterations:  2 Lines skipped:  2000 records collected:  1000  - latest recordID:  borrowAsset   0x374e4347a5abad8ebf70cbb9fde86005a3bfbeaeee444981120505b2e49f050d  - actions DB length:  3000  - latest record from:  2023-02-19 19:57:11 0 days ago
iterations:  3 Lines skipped:  3000 records collected:  1000  - latest recordID:  borrowAsset   0x4c7cd0a0fe188a8049ac8c28269d52106804986dfb9956dd84e7206633b56fd4+1  - actions DB length:  4000  - latest record from:  2023-02-19 17:26:35 0 days ago
iterations:  

In [982]:
actionsDb['transactionReadableAmount'] = actionsDb['actions_amount']/(10**(actionsDb['actions_pair_asset_decimals']))
actionsDb['transactionReadableAmount']=actionsDb['transactionReadableAmount'].astype('float')
actionsDb.head(2000)

,actions_id,actions_type,actions_amount,actions_timestamp,actions_block,actions_token_name,actions_root_address,actions_pair_asset_symbol,actions_pair_asset_decimals,actions_pair_collateral_symbol,actions_pair_collateral_decimals,DateTime,transactionReadableAmount
0,0xa36fce829a2866df6030c9e52d02c538b7f1ca3ba28a...,borrowAsset,0,1676866727,16667337,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-20 04:18:47,0.000000
1,0xa36fce829a2866df6030c9e52d02c538b7f1ca3ba28a...,addCollateral,7006979439076347667,1676866727,16667337,Governance OHM,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-20 04:18:47,7.006979
2,0x374e4347a5abad8ebf70cbb9fde86005a3bfbeaeee44...,borrowAsset,10000000000000000000000,1676865431,16667229,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-20 03:57:11,10000.000000
3,0x4c7cd0a0fe188a8049ac8c28269d52106804986dfb99...,borrowAsset,0,1676856395,16666484,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-20 01:26:35,0.000000
4,0x4c7cd0a0fe188a8049ac8c28269d52106804986dfb99...,addCollateral,6480000000000000000,1676856395,16666484,Governance OHM,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-20 01:26:35,6.480000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0x3ee4a9ec901ef2eca3503aaa33817f3c9a97a5726a4d...,borrowAsset,3933309226525890551441,1670209907,16115831,Frax,0x313899a86cf680f12112758aa09daf0fc75510d6,FRAX,18,gOHM,18,2022-12-05 03:11:47,3933.309227
996,0xd0de210e6724178599f45fda0bd06d7fcac6e8d587f0...,addCollateral,20008057754316253453,1670209835,16115825,Governance OHM,0x313899a86cf680f12112758aa09daf0fc75510d6,FRAX,18,gOHM,18,2022-12-05 03:10:35,20.008058
997,0x39003c14500ed2c34fdc8b74fd67ce93b94f41c97808...,borrowAsset,2000000000000000000000,1670205551,16115472,Frax,0x51a0a3849f1cc8822c2cbca4e0ce9ee0ba8e481d,FRAX,18,gOHM,18,2022-12-05 01:59:11,2000.000000
998,0x39003c14500ed2c34fdc8b74fd67ce93b94f41c97808...,addCollateral,2666844721390373735,1670205551,16115472,Governance OHM,0x51a0a3849f1cc8822c2cbca4e0ce9ee0ba8e481d,FRAX,18,gOHM,18,2022-12-05 01:59:11,2.666845


In [983]:
fraxActionsDb= pd.DataFrame()
fraxActionsDbTimed =pd.concat([actionsDb, oldFraxActions])
fraxActionsDbTimed.head(1000)

,actions_id,actions_type,actions_amount,actions_timestamp,actions_block,actions_token_name,actions_root_address,actions_pair_asset_symbol,actions_pair_asset_decimals,actions_pair_collateral_symbol,actions_pair_collateral_decimals,DateTime,transactionReadableAmount
0,0xa36fce829a2866df6030c9e52d02c538b7f1ca3ba28a...,borrowAsset,0,1676866727,16667337,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-20 04:18:47,0.000000
1,0xa36fce829a2866df6030c9e52d02c538b7f1ca3ba28a...,addCollateral,7006979439076347667,1676866727,16667337,Governance OHM,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-20 04:18:47,7.006979
2,0x374e4347a5abad8ebf70cbb9fde86005a3bfbeaeee44...,borrowAsset,10000000000000000000000,1676865431,16667229,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-20 03:57:11,10000.000000
3,0x4c7cd0a0fe188a8049ac8c28269d52106804986dfb99...,borrowAsset,0,1676856395,16666484,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-20 01:26:35,0.000000
4,0x4c7cd0a0fe188a8049ac8c28269d52106804986dfb99...,addCollateral,6480000000000000000,1676856395,16666484,Governance OHM,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-20 01:26:35,6.480000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0x3ee4a9ec901ef2eca3503aaa33817f3c9a97a5726a4d...,borrowAsset,3933309226525890551441,1670209907,16115831,Frax,0x313899a86cf680f12112758aa09daf0fc75510d6,FRAX,18,gOHM,18,2022-12-05 03:11:47,3933.309227
996,0xd0de210e6724178599f45fda0bd06d7fcac6e8d587f0...,addCollateral,20008057754316253453,1670209835,16115825,Governance OHM,0x313899a86cf680f12112758aa09daf0fc75510d6,FRAX,18,gOHM,18,2022-12-05 03:10:35,20.008058
997,0x39003c14500ed2c34fdc8b74fd67ce93b94f41c97808...,borrowAsset,2000000000000000000000,1670205551,16115472,Frax,0x51a0a3849f1cc8822c2cbca4e0ce9ee0ba8e481d,FRAX,18,gOHM,18,2022-12-05 01:59:11,2000.000000
998,0x39003c14500ed2c34fdc8b74fd67ce93b94f41c97808...,addCollateral,2666844721390373735,1670205551,16115472,Governance OHM,0x51a0a3849f1cc8822c2cbca4e0ce9ee0ba8e481d,FRAX,18,gOHM,18,2022-12-05 01:59:11,2.666845


In [984]:
fraxActionsDbTimedLength = len(fraxActionsDbTimed)
path =file+'/frax_borrows.csv'
fraxActionsDbTimed.to_csv(path, index = False)

path =file+'/frax_borrows_piped.csv'
fraxActionsDbTimed.to_csv(path, index = False, sep='|')

print(path, 'file saved,', fraxActionsDbTimedLength,' records at', (dt.datetime.now()) )


D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\gohmAsStablecoinCollateral/data/frax_borrows_piped.csv file saved, 484000  records at 2023-02-19 22:36:48.028935


In [985]:
fraxActionsDbTimed.head(10)

,actions_id,actions_type,actions_amount,actions_timestamp,actions_block,actions_token_name,actions_root_address,actions_pair_asset_symbol,actions_pair_asset_decimals,actions_pair_collateral_symbol,actions_pair_collateral_decimals,DateTime,transactionReadableAmount
0,0xa36fce829a2866df6030c9e52d02c538b7f1ca3ba28a...,borrowAsset,0,1676866727,16667337,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-20 04:18:47,0.000000
1,0xa36fce829a2866df6030c9e52d02c538b7f1ca3ba28a...,addCollateral,7006979439076347667,1676866727,16667337,Governance OHM,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-20 04:18:47,7.006979
2,0x374e4347a5abad8ebf70cbb9fde86005a3bfbeaeee44...,borrowAsset,10000000000000000000000,1676865431,16667229,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-20 03:57:11,10000.000000
3,0x4c7cd0a0fe188a8049ac8c28269d52106804986dfb99...,borrowAsset,0,1676856395,16666484,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-20 01:26:35,0.000000
4,0x4c7cd0a0fe188a8049ac8c28269d52106804986dfb99...,addCollateral,6480000000000000000,1676856395,16666484,Governance OHM,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-20 01:26:35,6.480000
5,0xbea1383c97aff27bcf1cbe8f19ba3938e16a0f04b45f...,borrowAsset,40000000000000000000000,1676837783,16664949,Frax,0x343ee80f82440a498e48ae186e066809750a055b,FRAX,18,gOHM,18,2023-02-19 20:16:23,40000.000000
6,0xbea1383c97aff27bcf1cbe8f19ba3938e16a0f04b45f...,addCollateral,20000000000000000000,1676837783,16664949,Governance OHM,0x343ee80f82440a498e48ae186e066809750a055b,FRAX,18,gOHM,18,2023-02-19 20:16:23,20.000000
7,0x15f93b52199fd785e5e2423777b99e80b93b5ac0ea6f...,borrowAsset,400000000000000000000,1676823143,16663746,Frax,0x4731223a7b5ac0c425f1f35d0bdf6d340a93a181,FRAX,18,gOHM,18,2023-02-19 16:12:23,400.000000
8,0x634c74994952e29a5c2a5cc59251aece8937130cdd40...,borrowAsset,10000000000000000000000,1676816711,16663213,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-19 14:25:11,10000.000000
9,0x0ae9fafdbef4514be9b8310a7ee390fdd6151f6cba75...,borrowAsset,10000000000000000000000,1676814035,16662993,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-02-19 13:40:35,10000.000000


In [986]:
#we might as well create all the DBs here and save ourselves time and grief
fraxBorrowsDbTimed = pd.DataFrame()
fraxBorrowsDbTimed = db.query("Select distinct * from fraxActionsDbTimed where actions_type = 'borrowAsset' and actions_pair_collateral_symbol= 'gOHM' and right(actions_id,2)<>'+1' ").df()

fraxDepositsDbTimed = pd.DataFrame()
fraxDepositsDbTimed =fraxBorrows = db.query("Select distinct * from fraxActionsDbTimed where actions_type = 'addCollateral' and actions_pair_collateral_symbol= 'gOHM' and right(actions_id,2)<>'+1' ").df()

fraxWithdrawsDbTimed = pd.DataFrame()
fraxWithdrawsDbTimed = db.query("Select distinct * from fraxActionsDbTimed where actions_type = 'removeCollateral' and actions_pair_collateral_symbol= 'gOHM' and right(actions_id,2)<>'+1' ").df()

In [987]:
fraxBorrowsDbTimed.head(1000)

,actions_id,actions_type,actions_amount,actions_timestamp,actions_block,actions_token_name,actions_root_address,actions_pair_asset_symbol,actions_pair_asset_decimals,actions_pair_collateral_symbol,actions_pair_collateral_decimals,DateTime,transactionReadableAmount
0,0x08cef0dcfa82fb386ec9948dadfaa9dba0da206c6523...,borrowAsset,60000000000000000000000,1671302279,16206258,Frax,0x68492881985afd26b04dcc4963ca5128ccd3dd9d,FRAX,18,gOHM,18,2022-12-17 18:37:59,60000.000000
1,0x52a22509d8397aec4b08a262c8e50ce0e9e810cc1981...,borrowAsset,10000000000000000000000,1671284591,16204790,Frax,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,FRAX,18,gOHM,18,2022-12-17 13:43:11,10000.000000
2,0x70031de3f701a9ac2d57f53fc400f51a6918cc09a037...,borrowAsset,12400000000000000000000,1671223127,16199693,Frax,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,FRAX,18,gOHM,18,2022-12-16 20:38:47,12400.000000
3,0x91ce97ab7feb2244dbbcc49a2bc94e8859c767eba7eb...,borrowAsset,15000000000000000000000,1671213215,16198877,Frax,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,FRAX,18,gOHM,18,2022-12-16 17:53:35,15000.000000
4,0x0622d462adf7b5db61999e4818580e948de56acee446...,borrowAsset,20000000000000000000000,1671212627,16198828,Frax,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,FRAX,18,gOHM,18,2022-12-16 17:43:47,20000.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,0x0edabc0905821baa5a53ed0e16332855fbe79f1c808a...,borrowAsset,2434316720914051302073,1672358843,16293894,Frax,0x5eef7b9b64220489d3c3e8843490f07e9d9efe91,FRAX,18,gOHM,18,2022-12-30 00:07:23,2434.316721
156,0x95ed0727a1e51bd46de1de35e2f12d794626d06ed8b8...,borrowAsset,7000000000000000000000,1672209647,16281520,Frax,0x6487d336bdec94d4714a4d217b15aa5c7b9dfa99,FRAX,18,gOHM,18,2022-12-28 06:40:47,7000.000000
157,0xf5e230c42166dac905d32b786935b4941a3c6036cc31...,borrowAsset,5000000000000000000000,1672058315,16268978,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2022-12-26 12:38:35,5000.000000
158,0xf56f494836f8ed5a0d135c33af6c3316582c39ab5c1a...,borrowAsset,2500000000000000000000,1671987815,16263129,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2022-12-25 17:03:35,2500.000000


In [988]:
fraxDepositsDbTimed.head(1000)

,actions_id,actions_type,actions_amount,actions_timestamp,actions_block,actions_token_name,actions_root_address,actions_pair_asset_symbol,actions_pair_asset_decimals,actions_pair_collateral_symbol,actions_pair_collateral_decimals,DateTime,transactionReadableAmount
0,0x2c0b67c5bac1a60d50eb090a8b6508241600a01960ff...,addCollateral,40000000000000000000,1671302171,16206249,Governance OHM,0x68492881985afd26b04dcc4963ca5128ccd3dd9d,FRAX,18,gOHM,18,2022-12-17 18:36:11,40.000000
1,0x89f288af5dbb01b2606695b10043efcca7250ffa4f27...,addCollateral,4002731028708112016,1671285023,16204826,Governance OHM,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,FRAX,18,gOHM,18,2022-12-17 13:50:23,4.002731
2,0x6426c8eaa10b310af812579129a15f523e5728b3b3b6...,addCollateral,1050000000000000000,1671270203,16203596,Governance OHM,0x3ea0f98d3c7c9bdf3c15acec5a0f040ed81af494,FRAX,18,gOHM,18,2022-12-17 09:43:23,1.050000
3,0xfe699cd2ba2b23392813283cf1d50c43db9ed094dedc...,addCollateral,4936656022539134425,1671223559,16199729,Governance OHM,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,FRAX,18,gOHM,18,2022-12-16 20:45:59,4.936656
4,0x746ec0ab5c5b7dd3e0ae34cdfe5f4a0c668bfed1aec8...,addCollateral,2000004944345133516,1671220619,16199487,Governance OHM,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,FRAX,18,gOHM,18,2022-12-16 19:56:59,2.000005
...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,0xd08091a3b2a72a2c2e4d621643bea1941a92c6033ae2...,addCollateral,65000000000000000000,1671974363,16262011,Governance OHM,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2022-12-25 13:19:23,65.000000
238,0x55ddc7798dfcc907498c2b034dc8a591b38880673bd9...,addCollateral,50000000000000000000,1671973295,16261922,Governance OHM,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2022-12-25 13:01:35,50.000000
239,0xc8087afe3b449835b235da90386ebedc6999c1c55dfb...,addCollateral,3989650402168860011,1671373499,16212156,Governance OHM,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,FRAX,18,gOHM,18,2022-12-18 14:24:59,3.989650
240,0x8bef55b3495f6ebde0fb7bf110ef9dacd6c148687ebf...,addCollateral,1309626013443408981,1674152663,16442620,Governance OHM,0x440c8ab91ba7c8763b3d9b3357dc1c0d13a148aa,FRAX,18,gOHM,18,2023-01-19 18:24:23,1.309626


In [989]:
fraxWithdrawsDbTimed.head(10)

,actions_id,actions_type,actions_amount,actions_timestamp,actions_block,actions_token_name,actions_root_address,actions_pair_asset_symbol,actions_pair_asset_decimals,actions_pair_collateral_symbol,actions_pair_collateral_decimals,DateTime,transactionReadableAmount
0,0xca92cec6cfb9ea2c2687f2f2ebe9b9e172708db2e8ad...,removeCollateral,60000000000000000000,1671280103,16204416,Governance OHM,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,FRAX,18,gOHM,18,2022-12-17 12:28:23,60.000000
1,0x0174fee4792718bba39e21032495709b16a0a69d326d...,removeCollateral,5000000000000000000,1671221507,16199560,Governance OHM,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,FRAX,18,gOHM,18,2022-12-16 20:11:47,5.000000
2,0x9e2127611c544f0a039cf02e3215b1a9386122eabfc3...,removeCollateral,2000000000000000000,1671220523,16199480,Governance OHM,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,FRAX,18,gOHM,18,2022-12-16 19:55:23,2.000000
3,0xbd38fb78f3d80b35eea7e709f7d05540a5e7dffb7114...,removeCollateral,20000000000000000000,1671218123,16199284,Governance OHM,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,FRAX,18,gOHM,18,2022-12-16 19:15:23,20.000000
4,0x9fc79141c68c74624c40d7f265b03469bb351087f040...,removeCollateral,50014515489685906711,1671200687,16197839,Governance OHM,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,FRAX,18,gOHM,18,2022-12-16 14:24:47,50.014515
5,0x9e6df83511869f0411adcc023319d247bd07fa56b287...,removeCollateral,21000000000000000000,1671200399,16197815,Governance OHM,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,FRAX,18,gOHM,18,2022-12-16 14:19:59,21.000000
6,0x233d2aed6a3cbd9716037a8fdabc5d11169ee0b46a2c...,removeCollateral,37660060241001404887,1671178019,16195957,Governance OHM,0x14975679e5f87c25fa2c54958e735a79b5b93043,FRAX,18,gOHM,18,2022-12-16 08:06:59,37.660060
7,0xbd194e2a12127aa42d659d90601aec52e7483338d15d...,removeCollateral,5029500000000000000,1671169607,16195260,Governance OHM,0x1f3f9c4d1de64fa4b4bc9df42b8ddd9b0354116d,FRAX,18,gOHM,18,2022-12-16 05:46:47,5.029500
8,0x54c22f8eacb8cbf0e645bd5e13109cd49dd3a863ec96...,removeCollateral,38000000000000000000,1671157439,16194251,Governance OHM,0x68492881985afd26b04dcc4963ca5128ccd3dd9d,FRAX,18,gOHM,18,2022-12-16 02:23:59,38.000000
9,0x67a3de6a34d0e5edaa93c1c07d5537441a64cbbfe9ba...,removeCollateral,200000000000000000000,1671058307,16186038,Governance OHM,0xca5df910f3e2226324707efe7fa6d03a0f821353,FRAX,18,gOHM,18,2022-12-14 22:51:47,200.000000


In [991]:
fraxActionsDbTimed = pd.concat([oldFraxBorrows,fraxBorrowsDbTimed]).drop_duplicates().reset_index(drop=True)
fraxActionsDbTimed.sort_values(['actions_id','actions_type','actions_amount','actions_timestamp','actions_block','actions_token_name','actions_root_address','DateTime'])
fraxActionsDbTimed.drop_duplicates(keep='first' )
fraxActionsDbTimed.head(100)

,actions_id,actions_type,actions_amount,actions_timestamp,actions_block,actions_token_name,actions_root_address,actions_pair_asset_symbol,actions_pair_asset_decimals,actions_pair_collateral_symbol,actions_pair_collateral_decimals,DateTime,transactionReadableAmount
0,0x08cef0dcfa82fb386ec9948dadfaa9dba0da206c6523...,borrowAsset,60000000000000000000000,1671302279,16206258,Frax,0x68492881985afd26b04dcc4963ca5128ccd3dd9d,FRAX,18,gOHM,18,2022-12-17 18:37:59,60000.0
1,0x52a22509d8397aec4b08a262c8e50ce0e9e810cc1981...,borrowAsset,10000000000000000000000,1671284591,16204790,Frax,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,FRAX,18,gOHM,18,2022-12-17 13:43:11,10000.0
2,0x70031de3f701a9ac2d57f53fc400f51a6918cc09a037...,borrowAsset,12400000000000000000000,1671223127,16199693,Frax,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,FRAX,18,gOHM,18,2022-12-16 20:38:47,12400.0
3,0x91ce97ab7feb2244dbbcc49a2bc94e8859c767eba7eb...,borrowAsset,15000000000000000000000,1671213215,16198877,Frax,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,FRAX,18,gOHM,18,2022-12-16 17:53:35,15000.0
4,0x0622d462adf7b5db61999e4818580e948de56acee446...,borrowAsset,20000000000000000000000,1671212627,16198828,Frax,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,FRAX,18,gOHM,18,2022-12-16 17:43:47,20000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0x177e04577b8e91124fb854181bbeedcefc660331a794...,borrowAsset,5000000000000000000000,1674065555,16435403,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-01-18 18:12:35,5000.0
96,0x40b4c7123b3dfa9a0a09ed60776eaf4c288442424a9f...,borrowAsset,350000000000000000000000,1674052199,16434297,Frax,0xee95e4cf086fc80df7ae0f39dfc9ea53a3eaadcb,FRAX,18,gOHM,18,2023-01-18 14:29:59,350000.0
97,0x6b7edfe0a07d8e657d79c56472a8cd294bd0e0ef44a7...,borrowAsset,5000000000000000000000,1674008315,16430653,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,18,gOHM,18,2023-01-18 02:18:35,5000.0
98,0xc7a3898a122851678d184b57c22f151fb4d8d4b100f5...,borrowAsset,1100000000000000000000,1673977079,16428064,Frax,0x440c8ab91ba7c8763b3d9b3357dc1c0d13a148aa,FRAX,18,gOHM,18,2023-01-17 17:37:59,1100.0


In [992]:
fraxActionsDbTimedLength = len(fraxActionsDbTimed)
path =file+'/frax_borrows.csv'
fraxActionsDbTimed.to_csv(path, index = False)

path =file+'/frax_borrows_piped.csv'
fraxActionsDbTimed.to_csv(path, index = False, sep='|')

print(path, 'file saved,', fraxActionsDbTimedLength,' records at', (dt.datetime.now()) )

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\gohmAsStablecoinCollateral/data/frax_borrows_piped.csv file saved, 160  records at 2023-02-19 22:37:35.574972


Collateral check

In [993]:
totalgOHMDepositedFrax = sum(fraxDepositsDbTimed['transactionReadableAmount'])
print('There has been a total of',round(totalgOHMDepositedFrax,2),  'gOHM deposited cumulatively as collateral  to date')

There has been a total of 7636.92 gOHM deposited cumulatively as collateral  to date


In [994]:
totalgOHMWithdrawnFrax = sum(fraxWithdrawsDbTimed['transactionReadableAmount'])
print('There has been a total of',round(totalgOHMWithdrawnFrax,2),  'gOHM withdrawn cumulatively as collateral  to date')

There has been a total of 6289.99 gOHM withdrawn cumulatively as collateral  to date


In [995]:
collateralBalance = totalgOHMDepositedFrax-totalgOHMWithdrawnFrax
print(totalgOHMDepositedFrax, ' - ',totalgOHMWithdrawnFrax,'=', collateralBalance, 'currently deposited as collateral in Silo' )

7636.9197428013185  -  6289.988456871596 = 1346.9312859297224 currently deposited as collateral in Silo


**********************************************************
MERGE DIFFERENT DBs

In [1032]:
#COLLATERALDEPOSITS
fraxDepositsDbPreMerge = db.query(
    "select distinct"
    " actions_id transaction_id"
    ", actions_timestamp transaction_timestamp"
    ", DateTime dateTime"
    ", actions_root_address transaction_from "
    ", actions_pair_collateral_symbol asset_symbol"
    ", transactionReadableAmount ReadableAmount "
    "from fraxDepositsDbTimed "
).df()
fraxDepositsDbPreMerge['protocol'] = 'Frax'
fraxDepositsDbPreMerge['action'] = 'depositCollateral'
fraxDepositsDbPreMerge.head(10)



,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,ReadableAmount,protocol,action
0,0x2c0b67c5bac1a60d50eb090a8b6508241600a01960ff...,1671302171,2022-12-17 18:36:11,0x68492881985afd26b04dcc4963ca5128ccd3dd9d,gOHM,40.000000,Frax,depositCollateral
1,0x89f288af5dbb01b2606695b10043efcca7250ffa4f27...,1671285023,2022-12-17 13:50:23,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,gOHM,4.002731,Frax,depositCollateral
2,0x6426c8eaa10b310af812579129a15f523e5728b3b3b6...,1671270203,2022-12-17 09:43:23,0x3ea0f98d3c7c9bdf3c15acec5a0f040ed81af494,gOHM,1.050000,Frax,depositCollateral
3,0xfe699cd2ba2b23392813283cf1d50c43db9ed094dedc...,1671223559,2022-12-16 20:45:59,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,gOHM,4.936656,Frax,depositCollateral
4,0x746ec0ab5c5b7dd3e0ae34cdfe5f4a0c668bfed1aec8...,1671220619,2022-12-16 19:56:59,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,gOHM,2.000005,Frax,depositCollateral
5,0x3b19215815f770647d3dc58f46b288c52521fb9c5dba...,1671220199,2022-12-16 19:49:59,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,gOHM,50.000000,Frax,depositCollateral
6,0x34b32f25104dc3ff1f73b424f4dd47b9ae520aa7ea6f...,1671219503,2022-12-16 19:38:23,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,gOHM,19.771750,Frax,depositCollateral
7,0xb756d3f20a4933253711c29af3733719925dcb4dad26...,1671217427,2022-12-16 19:03:47,0xbf23c4820def2ed7cab808bec348f7072035a289,gOHM,290.000000,Frax,depositCollateral
8,0x117ff749d2618d289a02a226ddc6cbd25ea816f3fa07...,1671213383,2022-12-16 17:56:23,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,gOHM,5.979708,Frax,depositCollateral
9,0xbbaf3bbd990a4c2ca590bd50e4639fdef25fb0737d45...,1671213191,2022-12-16 17:53:11,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,gOHM,27.955417,Frax,depositCollateral


In [1033]:
siloDepositEventsDbTimedClean.head(1)

,event,transaction_hash,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_market_id,transaction_asset_name,transaction_asset_symbol,transactionReadableAmount,transaction_asset_lastPriceUSD,transaction_amountUSD,dateTime,calDate
0,deposit,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,16651409,1676673263,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,Governance OHM,gOHM,48.922199,2838.193286,138354.126796,2023-02-17 22:34:23,2023-02-17


In [1034]:
siloDepositsDbPreMerge = db.query(
    "select distinct"
    " transaction_hash transaction_id "
    ", transaction_timestamp transaction_timestamp "
    ", dateTime "
    ", transaction_account_id transaction_from "
    ", transaction_asset_symbol asset_symbol "
    ", transactionReadableAmount ReadableAmount "
    "from siloDepositEventsDbTimedClean "
).df()
siloDepositsDbPreMerge['protocol'] = 'silo'
siloDepositsDbPreMerge['action'] = 'depositCollateral'
siloDepositsDbPreMerge.head(10)


,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,ReadableAmount,protocol,action
0,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,1676673263,2023-02-17 22:34:23,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,gOHM,48.922199,silo,depositCollateral
1,0x54cd3f5374780bceaffae0661e2688eeca47df22b8aa...,1676555939,2023-02-16 13:58:59,0xee95e4cf086fc80df7ae0f39dfc9ea53a3eaadcb,gOHM,50.749992,silo,depositCollateral
2,0xe5163797f0922e4bded89126a213296a8c3d270a39e2...,1676383883,2023-02-14 14:11:23,0xc0b1f200aa9546ce074397bed7438bb2ec033877,gOHM,0.180346,silo,depositCollateral
3,0xcc5e09e1e5c403b6925fa25f0410388de72ef1b45f30...,1676383523,2023-02-14 14:05:23,0xc0b1f200aa9546ce074397bed7438bb2ec033877,gOHM,0.663108,silo,depositCollateral
4,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,1676330339,2023-02-13 23:18:59,0xc0b1f200aa9546ce074397bed7438bb2ec033877,gOHM,0.892669,silo,depositCollateral
5,0x3069fa3e9834fd75b4eaae5d1ef1a7910b112ed68e5f...,1676330087,2023-02-13 23:14:47,0xc0b1f200aa9546ce074397bed7438bb2ec033877,gOHM,1.087799,silo,depositCollateral
6,0x17d879ec97730987652b5bcb00aaf6115b7dd232aaef...,1676292887,2023-02-13 12:54:47,0xc0b1f200aa9546ce074397bed7438bb2ec033877,gOHM,1.812154,silo,depositCollateral
7,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,1676069891,2023-02-10 22:58:11,0xa7daeb7fc210f934508c8c7ce833a2cdd6630fb3,gOHM,0.137814,silo,depositCollateral
8,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,1675938899,2023-02-09 10:34:59,0x51bc3dd10a9ee4946f961038cf5d50057e49eb77,gOHM,9.990000,silo,depositCollateral
9,0x204aee0831212791916d6ed3a2c24b5ac2a474743713...,1675923587,2023-02-09 06:19:47,0xb0399dac8f4d4100b49b2a7b3873481114229d18,gOHM,119.000000,silo,depositCollateral


DONT FORGET VESTA WHEN IT GETS FIXED

In [1035]:
gOHMCollateralDeposits = pd.concat([siloDepositsDbPreMerge,fraxDepositsDbPreMerge])
gOHMCollateralDeposits.head(1000)

,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,ReadableAmount,protocol,action
0,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,1676673263,2023-02-17 22:34:23,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,gOHM,48.922199,silo,depositCollateral
1,0x54cd3f5374780bceaffae0661e2688eeca47df22b8aa...,1676555939,2023-02-16 13:58:59,0xee95e4cf086fc80df7ae0f39dfc9ea53a3eaadcb,gOHM,50.749992,silo,depositCollateral
2,0xe5163797f0922e4bded89126a213296a8c3d270a39e2...,1676383883,2023-02-14 14:11:23,0xc0b1f200aa9546ce074397bed7438bb2ec033877,gOHM,0.180346,silo,depositCollateral
3,0xcc5e09e1e5c403b6925fa25f0410388de72ef1b45f30...,1676383523,2023-02-14 14:05:23,0xc0b1f200aa9546ce074397bed7438bb2ec033877,gOHM,0.663108,silo,depositCollateral
4,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,1676330339,2023-02-13 23:18:59,0xc0b1f200aa9546ce074397bed7438bb2ec033877,gOHM,0.892669,silo,depositCollateral
...,...,...,...,...,...,...,...,...
237,0xf79086c46e8db29afdfa2f26bc6f6e4f4ea9a58cdce3...,1675625747,2023-02-05 19:35:47,0x46535cbc3fa574c32ea392d4f89c518290597072,gOHM,28.311549,Frax,depositCollateral
238,0x2b4448fd953c5accbab7d852020be209da0f5eb83071...,1675381655,2023-02-02 23:47:35,0xc0e0de939f26e71683452467942aff455ca153e3,gOHM,5.440000,Frax,depositCollateral
239,0x06806270acee99df1ef398d1c7e82340e1451c9bb50f...,1673463803,2023-01-11 19:03:23,0x440c8ab91ba7c8763b3d9b3357dc1c0d13a148aa,gOHM,1.111000,Frax,depositCollateral
240,0x435c724feefb06c11e0c6166057452856d25685b4b97...,1671407015,2022-12-18 23:43:35,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,gOHM,3.962751,Frax,depositCollateral


In [1036]:
gOHMCollateralDepositsLength = len(gOHMCollateralDeposits)
path =file+'/gOHMCollateralDeposits.csv'
gOHMCollateralDeposits.to_csv(path, index = False)

path =file+'/gOHMCollateralDeposits_piped.csv'
gOHMCollateralDeposits.to_csv(path, index = False, sep='|')

print(path, 'file saved,', gOHMCollateralDepositsLength,' records at', (dt.datetime.now()) )

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\gohmAsStablecoinCollateral/data/gOHMCollateralDeposits_piped.csv file saved, 275  records at 2023-02-19 22:51:27.094452


In [1037]:
#ALL WITHDRAW EVENTS

In [1042]:
fraxWithdrawsDbPreMerge = db.query(
    "select distinct"
    " actions_id transaction_id"
    ", actions_timestamp transaction_timestamp"
    ", DateTime dateTime "
    ", actions_root_address transaction_from "
    ", actions_pair_collateral_symbol asset_symbol "
    ", transactionReadableAmount ReadableAmount "
    "from fraxWithdrawsDbTimed "
).df()
fraxWithdrawsDbPreMerge['protocol'] = 'Frax'
fraxWithdrawsDbPreMerge['action'] = 'withdrawCollateral'
fraxWithdrawsDbPreMerge.head(1000)

,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,ReadableAmount,protocol,action
0,0xca92cec6cfb9ea2c2687f2f2ebe9b9e172708db2e8ad...,1671280103,2022-12-17 12:28:23,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,gOHM,60.000000,Frax,withdrawCollateral
1,0x0174fee4792718bba39e21032495709b16a0a69d326d...,1671221507,2022-12-16 20:11:47,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,gOHM,5.000000,Frax,withdrawCollateral
2,0x9e2127611c544f0a039cf02e3215b1a9386122eabfc3...,1671220523,2022-12-16 19:55:23,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,gOHM,2.000000,Frax,withdrawCollateral
3,0xbd38fb78f3d80b35eea7e709f7d05540a5e7dffb7114...,1671218123,2022-12-16 19:15:23,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,gOHM,20.000000,Frax,withdrawCollateral
4,0x9fc79141c68c74624c40d7f265b03469bb351087f040...,1671200687,2022-12-16 14:24:47,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,gOHM,50.014515,Frax,withdrawCollateral
...,...,...,...,...,...,...,...,...
132,0x147ad8ff181623a8569f07f84ac4538d3f85aa74761b...,1674061211,2023-01-18 17:00:11,0xc0e0de939f26e71683452467942aff455ca153e3,gOHM,25.000000,Frax,withdrawCollateral
133,0xf3fea8b4ca84b5579566f5c0e117c0afbd5b9bf6a2c3...,1672535579,2023-01-01 01:12:59,0xffbd7b73e7b9e84e719f08082de6f3a39a12b862,gOHM,65.553667,Frax,withdrawCollateral
134,0x7060039897bd7b553401d3a98d76b101b548562825f6...,1671632807,2022-12-21 14:26:47,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,gOHM,50.000007,Frax,withdrawCollateral
135,0xb4453e19c0fd998a9d0a09e119ee09dba38828b503ac...,1671591707,2022-12-21 03:01:47,0x6a7dbffd83c6808651223efcaf7ea88161290e2b,gOHM,7.519936,Frax,withdrawCollateral


In [1043]:
siloWithdrawsDbPreMerge = db.query(
    "select distinct"
    " transaction_hash transaction_id "
    ", transaction_timestamp transaction_timestamp "
    ", dateTime "
    ", transaction_account_id transaction_from "
    ", transaction_asset_symbol asset_symbol "
    ", transactionReadableAmount ReadableAmount "
    "from siloWithdrawEventsDbTimedClean "
).df()
siloWithdrawsDbPreMerge['protocol'] = 'silo'
siloWithdrawsDbPreMerge['action'] = 'withdrawCollateral'
siloWithdrawsDbPreMerge.head(1000)


,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,ReadableAmount,protocol,action
0,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,1676474867,2023-02-15 15:27:47,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,gOHM,25.000039,silo,withdrawCollateral
1,0x85b689883076aa92af0a2356f8e4084610578bb48590...,1676472599,2023-02-15 14:49:59,0xb0399dac8f4d4100b49b2a7b3873481114229d18,gOHM,64.393035,silo,withdrawCollateral
2,0xd931b79cfc34df088f325e8adde9122af7ad64a0ba30...,1676391839,2023-02-14 16:23:59,0xb0399dac8f4d4100b49b2a7b3873481114229d18,gOHM,36.000000,silo,withdrawCollateral
3,0xd4f7aeb47e1e07903a3e9298762bc164764fffb2f973...,1676381987,2023-02-14 13:39:47,0xc0b1f200aa9546ce074397bed7438bb2ec033877,gOHM,0.883213,silo,withdrawCollateral
4,0xc7193a2aa216c65fe82652e85b4adc20a60b274bd070...,1676072123,2023-02-10 23:35:23,0xa7daeb7fc210f934508c8c7ce833a2cdd6630fb3,gOHM,0.137814,silo,withdrawCollateral
5,0xed87a5e09b9abe135be66d51493eca516d78aaf1d4d5...,1675924055,2023-02-09 06:27:35,0xb0399dac8f4d4100b49b2a7b3873481114229d18,gOHM,19.607121,silo,withdrawCollateral
6,0xcfa1e8a102e407470f7cc479576cceab5f872c1a1377...,1675737947,2023-02-07 02:45:47,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,gOHM,0.194798,silo,withdrawCollateral
7,0xb8682e4d0d5fe1297f21ff66347debdfc75c3956612d...,1674641375,2023-01-25 10:09:35,0xee1520f94f304e8d551cbf310fe214212e3ca34a,gOHM,20.000000,silo,withdrawCollateral
8,0xcc9db9a7563b473c4b8e4ca0f4f7fb90ca709b022469...,1673230019,2023-01-09 02:06:59,0x25fdda95197f804316f5ba8dbb66ac173ee19981,gOHM,2.252766,silo,withdrawCollateral
9,0x7c24d04318da39f08c7ba4f8d413a487133ce1b1f117...,1671741731,2022-12-22 20:42:11,0x96035b45bc0d22f0ed11283a91145fb001c48370,gOHM,13.520668,silo,withdrawCollateral


DONT FORGET VESTA

In [1044]:
gOHMCollateralWithdrawals = pd.concat([siloWithdrawsDbPreMerge,fraxWithdrawsDbPreMerge])
gOHMCollateralWithdrawals.head(1000)

,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,ReadableAmount,protocol,action
0,0xf4707edc29622f90030af624d1918ff5ae613cef90d8...,1676474867,2023-02-15 15:27:47,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,gOHM,25.000039,silo,withdrawCollateral
1,0x85b689883076aa92af0a2356f8e4084610578bb48590...,1676472599,2023-02-15 14:49:59,0xb0399dac8f4d4100b49b2a7b3873481114229d18,gOHM,64.393035,silo,withdrawCollateral
2,0xd931b79cfc34df088f325e8adde9122af7ad64a0ba30...,1676391839,2023-02-14 16:23:59,0xb0399dac8f4d4100b49b2a7b3873481114229d18,gOHM,36.000000,silo,withdrawCollateral
3,0xd4f7aeb47e1e07903a3e9298762bc164764fffb2f973...,1676381987,2023-02-14 13:39:47,0xc0b1f200aa9546ce074397bed7438bb2ec033877,gOHM,0.883213,silo,withdrawCollateral
4,0xc7193a2aa216c65fe82652e85b4adc20a60b274bd070...,1676072123,2023-02-10 23:35:23,0xa7daeb7fc210f934508c8c7ce833a2cdd6630fb3,gOHM,0.137814,silo,withdrawCollateral
...,...,...,...,...,...,...,...,...
132,0x147ad8ff181623a8569f07f84ac4538d3f85aa74761b...,1674061211,2023-01-18 17:00:11,0xc0e0de939f26e71683452467942aff455ca153e3,gOHM,25.000000,Frax,withdrawCollateral
133,0xf3fea8b4ca84b5579566f5c0e117c0afbd5b9bf6a2c3...,1672535579,2023-01-01 01:12:59,0xffbd7b73e7b9e84e719f08082de6f3a39a12b862,gOHM,65.553667,Frax,withdrawCollateral
134,0x7060039897bd7b553401d3a98d76b101b548562825f6...,1671632807,2022-12-21 14:26:47,0x0493bcca897a2248ae3cd4c66aea90dbfc7e13c6,gOHM,50.000007,Frax,withdrawCollateral
135,0xb4453e19c0fd998a9d0a09e119ee09dba38828b503ac...,1671591707,2022-12-21 03:01:47,0x6a7dbffd83c6808651223efcaf7ea88161290e2b,gOHM,7.519936,Frax,withdrawCollateral


In [1045]:
gOHMCollateralWithdrawalsLength = len(gOHMCollateralWithdrawals)
path =file+'/gOHMCollateralDeposits.csv'
gOHMCollateralWithdrawals.to_csv(path, index = False)

path =file+'/gOHMCollateralWithdrawals_piped.csv'
gOHMCollateralWithdrawals.to_csv(path, index = False, sep='|')

print(path, 'file saved,', gOHMCollateralWithdrawalsLength,' records at', (dt.datetime.now()) )

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\gohmAsStablecoinCollateral/data/gOHMCollateralWithdrawals_piped.csv file saved, 149  records at 2023-02-19 22:53:34.692979


In [1006]:
#BORROWS

In [1051]:
fraxBorrowsDbPreMerge = db.query(
    "select distinct"
    " actions_id transaction_id"
    ", actions_timestamp transaction_timestamp"
    ", DateTime as dateTime "
    ", actions_root_address transaction_from "
    ", 'FRAX' asset_symbol "
    ", transactionReadableAmount ReadableAmount "
    "from fraxBorrowsDbTimed "
).df()
fraxBorrowsDbPreMerge['protocol'] = 'Frax'
fraxBorrowsDbPreMerge['action'] = 'borrowStablecoin'
fraxBorrowsDbPreMerge.head(10)

,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,ReadableAmount,protocol,action
0,0x08cef0dcfa82fb386ec9948dadfaa9dba0da206c6523...,1671302279,2022-12-17 18:37:59,0x68492881985afd26b04dcc4963ca5128ccd3dd9d,FRAX,60000.0,Frax,borrowStablecoin
1,0x52a22509d8397aec4b08a262c8e50ce0e9e810cc1981...,1671284591,2022-12-17 13:43:11,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,FRAX,10000.0,Frax,borrowStablecoin
2,0x70031de3f701a9ac2d57f53fc400f51a6918cc09a037...,1671223127,2022-12-16 20:38:47,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,FRAX,12400.0,Frax,borrowStablecoin
3,0x91ce97ab7feb2244dbbcc49a2bc94e8859c767eba7eb...,1671213215,2022-12-16 17:53:35,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,FRAX,15000.0,Frax,borrowStablecoin
4,0x0622d462adf7b5db61999e4818580e948de56acee446...,1671212627,2022-12-16 17:43:47,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,FRAX,20000.0,Frax,borrowStablecoin
5,0x0d2211ab696b80c95abfc4a9674049cc5ca962a65512...,1671121715,2022-12-15 16:28:35,0x233e3017447e66d91e2cd969233adc5bb0524ec1,FRAX,3000.0,Frax,borrowStablecoin
6,0xa983a16f6c1d80c3213e10db6ecaa110d5b2165694d5...,1671058775,2022-12-14 22:59:35,0xca5df910f3e2226324707efe7fa6d03a0f821353,FRAX,67000.0,Frax,borrowStablecoin
7,0xa1adc9066c5b6ab4c3065cce6c7a4098192f39f97aeb...,1671058415,2022-12-14 22:53:35,0xca5df910f3e2226324707efe7fa6d03a0f821353,FRAX,83000.0,Frax,borrowStablecoin
8,0xeab5c258109dad23c2bac870caad513ecd7a9276236b...,1670940851,2022-12-13 14:14:11,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,FRAX,5000.0,Frax,borrowStablecoin
9,0x56a091abfc08a39e9b81a36c7211c25ba828925a6b82...,1670907155,2022-12-13 04:52:35,0xffbd7b73e7b9e84e719f08082de6f3a39a12b862,FRAX,69000.0,Frax,borrowStablecoin


In [1052]:
siloBorrowsDbPreMerge = db.query(
    "select distinct"
    " transaction_hash transaction_id "
    ", transaction_timestamp transaction_timestamp "
    ", dateTime "
    ", transaction_account_id transaction_from "
    ", 'XAI' asset_symbol "
    ", transactionReadableAmount ReadableAmount "
    "from siloBorrowEventsDbTimedClean "
).df()
siloBorrowsDbPreMerge['protocol'] = 'silo'
siloBorrowsDbPreMerge['action'] = 'borrowStablecoin'
siloBorrowsDbPreMerge.head(10)

,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,ReadableAmount,protocol,action
0,0x66fe10ed6dcb8b83bdb5b0c46cd321060734b9063bd5...,1676673563,2023-02-17 22:39:23,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,XAI,60500.000000,silo,borrowStablecoin
1,0xd9329978828295f4f091eaa77a6d8c388c5616bbe92c...,1676590307,2023-02-16 23:31:47,0x96035b45bc0d22f0ed11283a91145fb001c48370,XAI,0.600000,silo,borrowStablecoin
2,0x3b0f26e10c63a71c916e8a68bd1889d9bab38aeb62d7...,1676556011,2023-02-16 14:00:11,0xee95e4cf086fc80df7ae0f39dfc9ea53a3eaadcb,XAI,100000.000000,silo,borrowStablecoin
3,0xfa292276fc71f90e29ca90a7b8a2be9a4094f459fb99...,1676472911,2023-02-15 14:55:11,0x16ff8ad104fa6195260a6d4a3f311c5f120df77d,XAI,3000.000000,silo,borrowStablecoin
4,0xd5dc7147192591c7d19856d12e9278928932a5921653...,1676383655,2023-02-14 14:07:35,0xc0b1f200aa9546ce074397bed7438bb2ec033877,XAI,1000.000000,silo,borrowStablecoin
5,0x39c11cb01e34d9f8ca55a8820482eaf8a217033cdb6f...,1676375459,2023-02-14 11:50:59,0x16ff8ad104fa6195260a6d4a3f311c5f120df77d,XAI,1.920000,silo,borrowStablecoin
6,0xf1d5b9c7f23bd9935061e4d08734cbfba9b9a53bacd5...,1676370563,2023-02-14 10:29:23,0x339fd7c17ce670e7821f2f2ee94ddd49771483da,XAI,4000.000000,silo,borrowStablecoin
7,0x97811f7e7b7d202797093d44047ed986054be3dd6120...,1676330255,2023-02-13 23:17:35,0xc0b1f200aa9546ce074397bed7438bb2ec033877,XAI,2457.516074,silo,borrowStablecoin
8,0x3dba2dd3c913b399164efd71db73121c9c94c0096139...,1676328371,2023-02-13 22:46:11,0x46535cbc3fa574c32ea392d4f89c518290597072,XAI,26.400000,silo,borrowStablecoin
9,0xbcfc05398a874e98a80e823ffb016c119b06c7b07a7c...,1676292983,2023-02-13 12:56:23,0xc0b1f200aa9546ce074397bed7438bb2ec033877,XAI,3090.000000,silo,borrowStablecoin


DONT FORGET VESTA

In [1053]:
stablecoinsBorrowedWithGOHMCollateral = pd.concat([siloBorrowsDbPreMerge,fraxBorrowsDbPreMerge])
stablecoinsBorrowedWithGOHMCollateral.head(1000)

,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,ReadableAmount,protocol,action
0,0x66fe10ed6dcb8b83bdb5b0c46cd321060734b9063bd5...,1676673563,2023-02-17 22:39:23,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,XAI,60500.0,silo,borrowStablecoin
1,0xd9329978828295f4f091eaa77a6d8c388c5616bbe92c...,1676590307,2023-02-16 23:31:47,0x96035b45bc0d22f0ed11283a91145fb001c48370,XAI,0.6,silo,borrowStablecoin
2,0x3b0f26e10c63a71c916e8a68bd1889d9bab38aeb62d7...,1676556011,2023-02-16 14:00:11,0xee95e4cf086fc80df7ae0f39dfc9ea53a3eaadcb,XAI,100000.0,silo,borrowStablecoin
3,0xfa292276fc71f90e29ca90a7b8a2be9a4094f459fb99...,1676472911,2023-02-15 14:55:11,0x16ff8ad104fa6195260a6d4a3f311c5f120df77d,XAI,3000.0,silo,borrowStablecoin
4,0xd5dc7147192591c7d19856d12e9278928932a5921653...,1676383655,2023-02-14 14:07:35,0xc0b1f200aa9546ce074397bed7438bb2ec033877,XAI,1000.0,silo,borrowStablecoin
...,...,...,...,...,...,...,...,...
155,0xaaf548c6f478c7824a79855f43417f8ec997e45b4194...,1673408951,2023-01-11 03:49:11,0x783f3cad39b6528e192722527429bbbdaad01648,FRAX,10000.0,Frax,borrowStablecoin
156,0x63cf9d10b4f61a4f6e3825ccf2e0acfbdcb9e9e08df1...,1672855139,2023-01-04 17:58:59,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,60000.0,Frax,borrowStablecoin
157,0x4f5be22865aa1ec8139baef4966db502388f1c4c04d8...,1671907883,2022-12-24 18:51:23,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,FRAX,5000.0,Frax,borrowStablecoin
158,0x90815f3fde58d0fb4c794dbb4b1d2967f7417dbfe3d0...,1671625175,2022-12-21 12:19:35,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,FRAX,5000.0,Frax,borrowStablecoin


In [1054]:
stablecoinsBorrowedWithGOHMCollateralDepositsLength = len(stablecoinsBorrowedWithGOHMCollateralDeposits)
path =file+'/stablecoinsBorrowedWithGOHMCollateralDeposits.csv'
stablecoinsBorrowedWithGOHMCollateralDeposits.to_csv(path, index = False)

path =file+'/stablecoinsBorrowedWithGOHMCollateralDeposits_piped.csv'
stablecoinsBorrowedWithGOHMCollateralDeposits.to_csv(path, index = False, sep='|')
print(path, 'file saved,', fraxBorrowsWithGOHMCollateralLength,' records at', (dt.datetime.now()) )

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\gohmAsStablecoinCollateral/data/stablecoinsBorrowedWithGOHMCollateralDeposits_piped.csv file saved, 275  records at 2023-02-19 22:55:34.208777


**********************************************************
MERGE EVERYTHING


In [1055]:
stablecoinsBorrowedWithGOHMCollateralDeposits.head(1)

,actions_id,actions_timestamp,deposit_from,asset_symbol,depositReadableAmount,protocol,action
0,0x66fe10ed6dcb8b83bdb5b0c46cd321060734b9063bd5...,1676673563,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,XAI,60500.0,silo,borrowStablecoin


In [1056]:
stablecoinsBorrowedWithGOHMCollateralEvents = pd.DataFrame()
stablecoinsBorrowedWithGOHMCollateralEvents = pd.concat([gOHMCollateralDeposits,gOHMCollateralWithdrawals,stablecoinsBorrowedWithGOHMCollateral])
stablecoinsBorrowedWithGOHMCollateralEvents.head(1000)

,transaction_id,transaction_timestamp,dateTime,transaction_from,asset_symbol,ReadableAmount,protocol,action
0,0xecd6140bbb896413a25d34ca87f5503b56c444f1e582...,1676673263,2023-02-17 22:34:23,0xd33e76d21d32c21811593ae54e38f68df0cfe8ed,gOHM,48.922199,silo,depositCollateral
1,0x54cd3f5374780bceaffae0661e2688eeca47df22b8aa...,1676555939,2023-02-16 13:58:59,0xee95e4cf086fc80df7ae0f39dfc9ea53a3eaadcb,gOHM,50.749992,silo,depositCollateral
2,0xe5163797f0922e4bded89126a213296a8c3d270a39e2...,1676383883,2023-02-14 14:11:23,0xc0b1f200aa9546ce074397bed7438bb2ec033877,gOHM,0.180346,silo,depositCollateral
3,0xcc5e09e1e5c403b6925fa25f0410388de72ef1b45f30...,1676383523,2023-02-14 14:05:23,0xc0b1f200aa9546ce074397bed7438bb2ec033877,gOHM,0.663108,silo,depositCollateral
4,0xf5439a3f94551476ec68db02d6f577a2856d01f45691...,1676330339,2023-02-13 23:18:59,0xc0b1f200aa9546ce074397bed7438bb2ec033877,gOHM,0.892669,silo,depositCollateral
...,...,...,...,...,...,...,...,...
155,0xaaf548c6f478c7824a79855f43417f8ec997e45b4194...,1673408951,2023-01-11 03:49:11,0x783f3cad39b6528e192722527429bbbdaad01648,FRAX,10000.000000,Frax,borrowStablecoin
156,0x63cf9d10b4f61a4f6e3825ccf2e0acfbdcb9e9e08df1...,1672855139,2023-01-04 17:58:59,0xc0e0de939f26e71683452467942aff455ca153e3,FRAX,60000.000000,Frax,borrowStablecoin
157,0x4f5be22865aa1ec8139baef4966db502388f1c4c04d8...,1671907883,2022-12-24 18:51:23,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,FRAX,5000.000000,Frax,borrowStablecoin
158,0x90815f3fde58d0fb4c794dbb4b1d2967f7417dbfe3d0...,1671625175,2022-12-21 12:19:35,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,FRAX,5000.000000,Frax,borrowStablecoin


In [1057]:
stablecoinsBorrowedWithGOHMCollateralEventsLength = len(stablecoinsBorrowedWithGOHMCollateralEvents)
path =file+'/stablecoinsBorrowedWithGOHMCollateralEvents.csv'
stablecoinsBorrowedWithGOHMCollateralEvents.to_csv(path, index = False)

path =file+'/stablecoinsBorrowedWithGOHMCollateralEvents_piped.csv'
stablecoinsBorrowedWithGOHMCollateralEvents.to_csv(path, index = False, sep='|')
print(path, 'file saved,', stablecoinsBorrowedWithGOHMCollateralEventsLength,' records at', (dt.datetime.now()) )

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\gohmAsStablecoinCollateral/data/stablecoinsBorrowedWithGOHMCollateralEvents_piped.csv file saved, 610  records at 2023-02-19 22:55:41.959565


FRAX DAILY SNAPS

In [1058]:
pairSnapshotsListLength=0
pairSnapshotsTicker = 0
Snapshots = pd.DataFrame()
pairSnapshotsDb = pd.DataFrame()
pairSnapshotsListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit= False
while exit==False:
    #listLength=0
    skipValue = (pairSnapshotsTicker)*(1000)
    pairsQuery = frax.Query.pairPerDays(
         orderBy='id'
        ,orderDirection='desc'
        ,first=1000
        ,skipValue = skipValue
        #,where = [frax.Query.pairs.token.id == '0x66bf36dba79d4606039f04b32946a260bcd3ff52']
        ,where={ 'pair': '0x66bf36dba79d4606039f04b32946a260bcd3ff52' }
        #,where = {'positions.pairid':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}
    #{'users_positions_pair_id':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}

    )
    Snapshots= sg.query_df([
    pairsQuery.id
    ,pairsQuery.pair.collateral.symbol
    ,pairsQuery.pair.collateral.decimals
    ,pairsQuery.pair.asset.symbol
    ,pairsQuery.pair.asset.decimals
    ,pairsQuery.feeToProtocolRate
    ,pairsQuery.exchangeRate
    ,pairsQuery.totalAssetAmount
    ,pairsQuery.totalAssetShare
    ,pairsQuery.totalCollateral
    ,pairsQuery.totalBorrowAmount
    ,pairsQuery.totalBorrowShare
    ,pairsQuery.interestPerSecond
    ,pairsQuery.utilization
    ,pairsQuery.totalFeesAmount
    ,pairsQuery.totalFeesShare
    ,pairsQuery.lastAccrued
    ,pairsQuery.block
    ,pairsQuery.timestamp
    ,pairsQuery.totalAssetValue
    ,pairsQuery.totalCollateralValue
    ,pairsQuery.totalBorrowValue
    ,pairsQuery.pair
    ]
    )

    Snapshots.sort_values(by=['pairPerDays_timestamp'])

    pairSnapshotsDb=pd.concat([pairSnapshotsDb, Snapshots])
    recordTimestamp1 = pairSnapshotsDb.iat[pairSnapshotsTicker,14]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    pairSnapshotsDbLength = len(pairSnapshotsDb)
    pairSnapshotsListLength = len(Snapshots)
    recordID = pairSnapshotsDb.iat[pairSnapshotsTicker,0]
    Snapshot = pairSnapshotsDb.iat[pairSnapshotsTicker,1]
    print("iterations: ", pairSnapshotsTicker, "Lines skipped: ",skipValue, "records collected: ", pairSnapshotsListLength, " - latest recordID: ",Snapshot, " ", recordID , " - pairSnapshots DB length: ", pairSnapshotsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", pairSnapshotsTicker, "records collected: ", pairSnapshotsListLength, " - userdb length: ", pairSnapshotsDbLength)
    if daysAgo>daysLimit: exit =True
    if pairSnapshotsListLength<1000: exit =True
    pairSnapshotsTicker = pairSnapshotsTicker+1

print("Done. Total Records collected: ", pairSnapshotsDbLength)

iterations:  0 Lines skipped:  0 records collected:  84  - latest recordID:  gOHM   0x66bf36dba79d4606039f04b32946a260bcd3ff52-Recent  - pairSnapshots DB length:  84  - latest record from:  1969-12-31 16:00:00 19408 days ago
Done. Total Records collected:  84


In [1059]:
pairSnapshotsDb.head(10)

,pairPerDays_id,pairPerDays_pair_collateral_symbol,pairPerDays_pair_collateral_decimals,pairPerDays_pair_asset_symbol,pairPerDays_pair_asset_decimals,pairPerDays_feeToProtocolRate,pairPerDays_exchangeRate,pairPerDays_totalAssetAmount,pairPerDays_totalAssetShare,pairPerDays_totalCollateral,...,pairPerDays_pair_collateral_id,pairPerDays_pair_oracleMultiplyAddress_id,pairPerDays_pair_oracleDivideAddress_id,pairPerDays_pair_rateContract_id,pairPerDays_pair_maxLTV,pairPerDays_pair_liquidationFee,pairPerDays_pair_maturity,pairPerDays_pair_pauseStatus,pairPerDays_pair_lenderWhitelistActive,pairPerDays_pair_borrowerWhitelistActive
0,0x66bf36dba79d4606039f04b32946a260bcd3ff52-Recent,gOHM,18,FRAX,18,0,353138047538614,5035697693045935634325418,4966638858282894118426824,3049199062322648903525,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False
1,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19408,gOHM,18,FRAX,18,0,353138047538614,5035697693045935634325418,4966638858282894118426824,3049199062322648903525,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False
2,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19407,gOHM,18,FRAX,18,0,354538364355376,5035386265636883114384206,4966638858282894118426824,3035712082883572555858,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False
3,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19406,gOHM,18,FRAX,18,0,353906299486322,5034722075873021602597750,4967011772748759733227735,3015712082883572555858,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False
4,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19405,gOHM,18,FRAX,18,0,350112376599804,4698486647811909309719977,4636381316961784483066425,2938561823965620072232,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False
5,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19404,gOHM,18,FRAX,18,0,363785846961333,4737561509933966578859758,4675953474727265989294250,2926144943608541855178,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False
6,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19403,gOHM,18,FRAX,18,0,346065938064754,4696443773732938768853377,4636381316961784483066425,2926505753609477928993,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False
7,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19402,gOHM,18,FRAX,18,0,355065538289602,4675315360621293243183076,4616545966840300870160156,2962449892162877274570,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False
8,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19401,gOHM,18,FRAX,18,0,361722603166047,5214478754084957077245512,5149945278165210104338772,2920785897200163201938,...,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False
9,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19400,gOHM,

In [1070]:
pairSnapshotsDb['pairPerDays_totalCollateralReadable'] = pairSnapshotsDb['pairPerDays_totalCollateral']/(10**(pairSnapshotsDb['pairPerDays_pair_collateral_decimals']))
pairSnapshotsDb['pairPerDays_totalCollateralReadable']=pairSnapshotsDb['pairPerDays_totalCollateralReadable'].astype('float')

pairSnapshotsDb['pairPerDays_totalAssetAmountReadable'] = pairSnapshotsDb['pairPerDays_totalAssetAmount']/(10**(pairSnapshotsDb['pairPerDays_pair_asset_decimals']))
pairSnapshotsDb['pairPerDays_totalAssetAmountReadable']=pairSnapshotsDb['pairPerDays_totalAssetAmountReadable'].astype('float')

pairSnapshotsDb['pairPerDays_totalBorrowAmountReadable'] = pairSnapshotsDb['pairPerDays_totalBorrowAmount']/(10**(pairSnapshotsDb['pairPerDays_pair_asset_decimals']))
pairSnapshotsDb['pairPerDays_totalBorrowAmountReadable']=pairSnapshotsDb['pairPerDays_totalBorrowAmountReadable'].astype('float')


pairSnapshotsDb['dateTime'] =(pd.to_datetime(pairSnapshotsDb['pairPerDays_timestamp'],unit='s'))

pairSnapshotsDb.drop_duplicates(keep ="first",inplace = True)


pairSnapshotsDb.head(2000)

,pairPerDays_id,pairPerDays_pair_collateral_symbol,pairPerDays_pair_collateral_decimals,pairPerDays_pair_asset_symbol,pairPerDays_pair_asset_decimals,pairPerDays_feeToProtocolRate,pairPerDays_exchangeRate,pairPerDays_totalAssetAmount,pairPerDays_totalAssetShare,pairPerDays_totalCollateral,...,pairPerDays_pair_maxLTV,pairPerDays_pair_liquidationFee,pairPerDays_pair_maturity,pairPerDays_pair_pauseStatus,pairPerDays_pair_lenderWhitelistActive,pairPerDays_pair_borrowerWhitelistActive,pairPerDays_totalCollateralReadable,pairPerDays_totalAssetAmountReadable,dateTime,pairPerDays_totalBorrowAmountReadable
0,0x66bf36dba79d4606039f04b32946a260bcd3ff52-Recent,gOHM,18,FRAX,18,0,353138047538614,5035697693045935634325418,4966638858282894118426824,3049199062322648903525,...,75000,10000,0,False,False,False,3049.199062,5.035698e+06,2023-02-20 04:18:47,4.201392e+06
1,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19408,gOHM,18,FRAX,18,0,353138047538614,5035697693045935634325418,4966638858282894118426824,3049199062322648903525,...,75000,10000,0,False,False,False,3049.199062,5.035698e+06,2023-02-20 04:18:47,4.201392e+06
2,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19407,gOHM,18,FRAX,18,0,354538364355376,5035386265636883114384206,4966638858282894118426824,3035712082883572555858,...,75000,10000,0,False,False,False,3035.712083,5.035386e+06,2023-02-19 20:16:23,4.191081e+06
3,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19406,gOHM,18,FRAX,18,0,353906299486322,5034722075873021602597750,4967011772748759733227735,3015712082883572555858,...,75000,10000,0,False,False,False,3015.712083,5.034722e+06,2023-02-18 22:49:59,4.129513e+06
4,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19405,gOHM,18,FRAX,18,0,350112376599804,4698486647811909309719977,4636381316961784483066425,2938561823965620072232,...,75000,10000,0,False,False,False,2938.561824,4.698487e+06,2023-02-17 23:41:35,4.018403e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19330,gOHM,18,FRAX,18,0,416982985435782,3000265799749067769553552,2999681610314728238726349,1702267776392927486576,...,75000,10000,0,False,False,False,1702.267776,3.000266e+06,2022-12-04 23:12:59,1.642246e+06
80,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19329,gOHM,18,FRAX,18,0,435238970363534,3000407417762830870845360,2999931610314728238726349,1460557767872656691963,...,75000,10000,0,False,False,False,1460.557768,3.000407e+06,2022-12-03 23:30:59,1.358687e+06
81,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19328,gOHM,18,FRAX,18,0,424429606966213,3000316417810236657492429,2999931610314728238726349,1298896246642712535146,...,75000,10000,0,False,False,False,1298.896247,3.000316e+06,2022-12-02 23:55:59,1.287539e+06
82,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19327,gOHM,18,FRAX,18,0,455232217741157,1000250018374254924219488,1000177511746822029696096,116544304370705257,...,75000,10000,0,False,False,False,0.116544,1.000250e+06,2022-12-01 21:30:47,1.821276e+02


In [1071]:
fraxPairSnapshotsLength = len(pairSnapshotsDb)
path =file+'/fraxPairSnapshotsLength.csv'
pairSnapshotsDb.to_csv(path, index = False)

path =file+'/fraxPairSnapshotsLength_piped.csv'
pairSnapshotsDb.to_csv(path, index = False, sep='|')
print(path, 'file saved,', fraxPairSnapshotsLength,' records at', (dt.datetime.now()) )

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\gohmAsStablecoinCollateral/data/fraxPairSnapshotsLength_piped.csv file saved, 84  records at 2023-02-19 23:04:31.545266


In [1085]:
fraxCollateral = pairSnapshotsDb.iat[pairSnapshotsTicker,37]
fraxCollateral

3049.199062322649

In [1084]:
siloCollateralBalance

182.3034925748639

In [1082]:
totalBorrowedUSDFrax= pairSnapshotsDb.iat[pairSnapshotsTicker,40]
totalBorrowedUSDFrax

4201392.002732461

In [1089]:
collateralGohmLevels = pd.DataFrame({
    'protocol':['Frax','vesta','silo'],
    'collateralBalance':[fraxCollateral,0,siloCollateralBalance],
    'totalBorrowed':[totalBorrowedUSDFrax,0,totalBorrowedUSDSilo]
})
collateralGohmLevels.head(10)

,protocol,collateralBalance,totalBorrowed
0,Frax,3049.199062,4.201392e+06
1,vesta,0.000000,0.000000e+00
2,silo,182.303493,5.645797e+05


In [1090]:
collateralGohmLevelsLength = len(collateralGohmLevels)
path =file+'/collateralGohmLevels.csv'
collateralGohmLevels.to_csv(path, index = False)

path =file+'/collateralGohmLevels_piped.csv'
collateralGohmLevels.to_csv(path, index = False, sep='|')
print(path, 'file saved,', collateralGohmLevelsLength,' records at', (dt.datetime.now()) )

D:\Dropbox\Documents\Edgecaser LLC\Projects\Olympus\Econohmetrics\gohmAsStablecoinCollateral/data/collateralGohmLevels_piped.csv file saved, 3  records at 2023-02-19 23:11:36.967639


In [1091]:
totalBorrowedUSDSilo

564579.7034215739